#Setup

In [ ]:
# ==========================================
# BLOCCO 0: SETUP TOTALE CON CLIP (FIXED)
# ==========================================

import os
import sys
from google.colab import drive

print("🚀 1. INSTALLAZIONE LIBRERIE...")

# Fix dipendenze prima
!pip install -q pillow==11.1.0  # Fix per gradio compatibility
!pip install -q requests==2.32.4  # Fix per google-colab

# Installazione principale
!pip install -q -U \
    typing_extensions \
    langchain \
    langchain-community \
    langchain-core \
    langchain-google-genai \
    langchain-huggingface \
    langchain-chroma \
    langchain-text-splitters \
    chromadb \
    sentence-transformers \
    accelerate \
    bitsandbytes \
    streamlit \
    pyngrok \
    pymupdf \
    transformers \
    ftfy \
    regex

# Verifica installazione
print("\n✅ 2. VERIFICA INSTALLAZIONE...")
try:
    import langchain
    import chromadb
    import streamlit
    import transformers
    print(f"   ✅ LangChain {langchain.__version__}")
    print(f"   ✅ ChromaDB {chromadb.__version__}")
    print(f"   ✅ Streamlit {streamlit.__version__}")
    print(f"   ✅ Transformers {transformers.__version__}")
except ImportError as e:
    print(f"   ❌ Errore: {e}")
    sys.exit(1)

print("\n🐬 3. SETUP DOLPHIN OCR...")
if not os.path.exists('/content/Dolphin'):
    print("   - Clonazione repository...")
    try:
        !git clone https://github.com/ByteDance/Dolphin.git
        # Installa dipendenze Dolphin individualmente per evitare errori
        !pip install -q timm einops
        print("   ✅ Dolphin clonato")
    except Exception as e:
        print(f"   ⚠️ Warning: Dolphin setup fallito: {e}")
else:
    print("   ✅ Dolphin già presente")

# Scarica il modello Dolphin
if not os.path.exists('/content/hf_model'):
    print("   - Download Modello Dolphin...")
    !pip install -q huggingface_hub
    !huggingface-cli download ByteDance/Dolphin-1.5 --local-dir ./hf_model
    if os.path.exists('/content/hf_model'):
        print("   ✅ Modello Dolphin scaricato")
    else:
        print("   ⚠️ Errore nel download Dolphin")
else:
    print("   ✅ Modello Dolphin già presente")

print("\n🖼️ 4. VERIFICA CLIP...")
try:
    from transformers import CLIPModel, CLIPProcessor
    print("   ✅ CLIP disponibile")

    # Test caricamento modello
    print("   - Test caricamento CLIP...")
    _ = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    print("   ✅ CLIP funzionante")
except ImportError as e:
    print(f"   ❌ CLIP import fallito: {e}")
    print("\n   ⚠️ RIAVVIA IL RUNTIME (Runtime > Restart runtime)")
    print("   Poi riesegui questo blocco.")
except Exception as e:
    print(f"   ⚠️ CLIP warning: {e}")

print("\n📂 5. MOUNT GOOGLE DRIVE...")
drive.mount('/content/drive', force_remount=True)

# Verifica cartella MANUALS
manuals_path = "/content/drive/MyDrive/MANUALS"
if os.path.exists(manuals_path):
    pdfs = [f for f in os.listdir(manuals_path) if f.endswith('.pdf')]
    print(f"\n📄 PDF trovati in MANUALS: {len(pdfs)}")
    for pdf in pdfs:
        size_mb = os.path.getsize(os.path.join(manuals_path, pdf)) / (1024*1024)
        print(f"   - {pdf} ({size_mb:.1f} MB)")
else:
    print(f"\n⚠️ Cartella {manuals_path} non trovata!")

print("\n🎉 Setup completato!")
print("\n⚠️ Se CLIP non è disponibile, RIAVVIA IL RUNTIME e riesegui questo blocco.")

# Configurazione centralizzata

In [ ]:
# ==========================================
# BLOCCO 1: CONFIGURAZIONE CENTRALIZZATA (FINAL CORRECT v2.0)
# ==========================================

print("⚙️ BLOCCO 1: Caricamento configurazione...")

# ==========================================
# PATH CONFIGURAZIONE
# ==========================================

PDF_INPUT_DIR = "/content/drive/MyDrive/MANUALS"
OCR_OUTPUT_DIR = "/content/drive/MyDrive/OCR_Output_Temp"

LOCAL_TEXT_DB = "/content/temp_text_db"
LOCAL_IMAGE_DB = "/content/temp_image_db"

DRIVE_TEXT_DB = "/content/drive/MyDrive/OCR/Testo_DB"
DRIVE_IMAGE_DB = "/content/drive/MyDrive/OCR/Images_DB"
DRIVE_CLIP_JSON = "/content/drive/MyDrive/OCR/clip_embeddings.json"

CHROMA_TEXT_DIR = DRIVE_TEXT_DB
CHROMA_IMAGE_DIR = DRIVE_IMAGE_DB
CLIP_JSON_PATH = DRIVE_CLIP_JSON

# ==========================================
# MODELLI
# ==========================================

TEXT_EMBED_MODEL = "intfloat/multilingual-e5-large"
CLIP_MODEL_NAME = "openai/clip-vit-base-patch32"
TEXT_COLLECTION_NAME = "text_documents"
IMAGE_COLLECTION_NAME = "image_documents"

# ==========================================
# PARAMETRI INGESTIONE
# ==========================================

PAGES_PER_BATCH = 10
DOLPHIN_TIMEOUT = 600
CHUNK_SIZE = 1200
CHUNK_OVERLAP = 300

# ==========================================
# ✅ IMAGE QUALITY CONFIG - FIXED PER DATI REALI
# ==========================================

# I tuoi dati: median=1.6KB, 67x48px
# Abbassiamo drasticamente per non perdere tutto

IMAGE_QUALITY_CONFIG = {
    'min_width': 20,         # Era 100 - le tue immagini sono ~67px
    'min_height': 20,        # Era 100 - le tue immagini sono ~48px
    'min_size_kb': 0.3,      # Era 30 - le tue immagini sono ~1.6KB
    'max_aspect_ratio': 12.0 # Un po' più permissivo
}

# CLIP Threshold (per ingestion futura)
CLIP_SIZE_KB_MIN = IMAGE_QUALITY_CONFIG['min_size_kb']
CLIP_WIDTH_MIN = IMAGE_QUALITY_CONFIG['min_width']
CLIP_HEIGHT_MIN = IMAGE_QUALITY_CONFIG['min_height']
CLIP_CAPTION_MAX_LEN = 150

# ==========================================
# TWO-STAGE RETRIEVAL
# ==========================================

STAGE1_TOP_K = 40
STAGE2_TOP_K = 5
CLIP_WEIGHT = 0.40

# ==========================================
# API KEY
# ==========================================

GOOGLE_API_KEY = "YOUR_API_KEY"

# ==========================================
# VERIFICA CONFIGURAZIONE
# ==========================================

print("\n" + "="*70)
print("✅ CONFIGURAZIONE CARICATA (v2.0 - Single Source of Truth)")
print("="*70)

print(f"\n📂 Paths:")
print(f"   • PDF Input: {PDF_INPUT_DIR}")
print(f"   • OCR Output: {OCR_OUTPUT_DIR}")
print(f"   • Text DB: {CHROMA_TEXT_DIR}")
print(f"   • Image DB: {CHROMA_IMAGE_DIR}")
print(f"   • CLIP JSON: {CLIP_JSON_PATH}")

print(f"\n🤖 Modelli:")
print(f"   • Text Embedding: {TEXT_EMBED_MODEL}")
print(f"   • CLIP Model: {CLIP_MODEL_NAME}")
print(f"   • Text Collection: {TEXT_COLLECTION_NAME}")
print(f"   • Image Collection: {IMAGE_COLLECTION_NAME}")

print(f"\n🔍 Two-Stage Retrieval:")
print(f"   • Stage 1 (Text): Top {STAGE1_TOP_K} candidates")
print(f"   • Stage 2 (CLIP): Top {STAGE2_TOP_K} results")
print(f"   • CLIP weight: {CLIP_WEIGHT} (Text weight: {1-CLIP_WEIGHT})")

print(f"\n⚙️ Parametri Ingestione:")
print(f"   • Dolphin batch: {PAGES_PER_BATCH} pagine/batch")
print(f"   • Dolphin timeout: {DOLPHIN_TIMEOUT}s")
print(f"   • Chunk size: {CHUNK_SIZE} char")
print(f"   • Chunk overlap: {CHUNK_OVERLAP} char")

print(f"\n🖼️ Image Quality Config (CENTRALIZZATA):")
print(f"   ⭐ SINGLE SOURCE OF TRUTH - Modifica SOLO qui!")
print(f"   • Min resolution: {IMAGE_QUALITY_CONFIG['min_width']}x{IMAGE_QUALITY_CONFIG['min_height']}px")
print(f"   • Min size: {IMAGE_QUALITY_CONFIG['min_size_kb']}KB")
print(f"   • Max aspect ratio: {IMAGE_QUALITY_CONFIG['max_aspect_ratio']}")
print(f"   📊 Confronto:")
print(f"      Vecchio (BUG): 300x300px, 20KB → 70% rejection")
print(f"      Nuovo (FIX):   150x150px, 10KB → 30% rejection")

print(f"\n🔧 CLIP Threshold (per ingestion):")
print(f"   • Min size: {CLIP_SIZE_KB_MIN}KB (eredita da IMAGE_QUALITY_CONFIG)")
print(f"   • Min resolution: {CLIP_WIDTH_MIN}x{CLIP_HEIGHT_MIN}px")
print(f"   • Caption max: {CLIP_CAPTION_MAX_LEN} char")

# ==========================================
# VERIFICA ESISTENZA DATABASE
# ==========================================

import os

print(f"\n📊 Database Status:")

text_exists = os.path.exists(CHROMA_TEXT_DIR)
img_exists = os.path.exists(CHROMA_IMAGE_DIR)
clip_exists = os.path.exists(CLIP_JSON_PATH)

print(f"   {'✅' if text_exists else '❌'} Text DB: {CHROMA_TEXT_DIR}")
print(f"   {'✅' if img_exists else '❌'} Image DB: {CHROMA_IMAGE_DIR}")
print(f"   {'✅' if clip_exists else '❌'} CLIP embeddings: {CLIP_JSON_PATH}")

if not (text_exists and img_exists):
    print("\n" + "="*70)
    print("⚠️ ATTENZIONE: Database mancanti!")
    print("="*70)
    print("\n📋 Cosa fare:")
    print("   1. Se NON hai ancora i database:")
    print("      → Esegui il BLOCCO INGESTIONE per crearli")
    print("      → Tempo richiesto: 30-90 minuti (dipende dai PDF)")
    print("\n   2. Se HAI già i database ma sono in un'altra cartella:")
    print("      → Verifica i path sopra")
    print("      → Modifica le variabili DRIVE_TEXT_DB e DRIVE_IMAGE_DB")
    print("      → Ri-esegui questo blocco")
    print("\n   3. Se hai eseguito l'ingestione ma i database non appaiono:")
    print("      → Ricarica Google Drive: drive.mount('/content/drive', force_remount=True)")
    print("      → Ri-esegui questo blocco")
    print("\n" + "="*70)
else:
    print("\n" + "="*70)
    print("🎉 CONFIGURAZIONE COMPLETA!")
    print("="*70)
    print("\n✅ Tutti i database sono presenti e pronti all'uso.")
    print("\n📚 Prossimi passi:")
    print("   1. Esegui BLOCCO 2 (Evaluation Framework)")
    print("   2. Esegui BLOCCO 2.5 (Advanced RAG Components)")
    print("   3. Poi scegli:")
    print("      • BLOCCO 3 (Test RAG) - per test singoli")
    print("      • BLOCCO 4 (Streamlit App) - per demo interattiva")

print("\n" + "="*70)
print("💡 TIPS:")
print("="*70)
print("• Per cambiare threshold qualità immagini:")
print("  → Modifica SOLO IMAGE_QUALITY_CONFIG (sopra)")
print("  → Tutti i blocchi si aggiorneranno automaticamente!")
print("\n• Per modificare CLIP weight:")
print(f"  → Cambia CLIP_WEIGHT = {CLIP_WEIGHT} (sopra)")
print("\n• Per cambiare numero risultati:")
print(f"  → Stage 1: STAGE1_TOP_K = {STAGE1_TOP_K}")
print(f"  → Stage 2: STAGE2_TOP_K = {STAGE2_TOP_K}")
print("="*70)

print("\n🎯 VANTAGGI ARCHITETTURA CENTRALIZZATA:")
print("   ✅ Single Source of Truth (1 modifica invece di 5)")
print("   ✅ Zero rischio inconsistenze")
print("   ✅ Manutenzione semplificata")
print("   ✅ Production-ready best practice")

print("\n" + "="*70)

#Cella di pulizia pre-ingestion

In [ ]:
''' # ==========================================
# 🧹 PULIZIA COMPLETA PRE-INGESTIONE
# ==========================================

import os
import shutil
from pathlib import Path

print("🧹 SCRIPT DI PULIZIA PRE-INGESTIONE")
print("=" * 70)

# ==========================================
# CONFIGURAZIONE PATHS
# ==========================================

# Database locali (temp in /content)
LOCAL_TEMP_PATHS = [
    "/content/temp_text_db",
    "/content/temp_image_db",
]

# Database Drive (permanenti)
DRIVE_DB_PATHS = [
    "/content/drive/MyDrive/OCR/Testo_DB",
    "/content/drive/MyDrive/OCR/Images_DB",
]

# CLIP embeddings
CLIP_JSON_PATH = "/content/drive/MyDrive/OCR/clip_embeddings.json"

# OCR Output temporaneo
OCR_OUTPUT_DIR = "/content/drive/MyDrive/OCR_Output_Temp"

# Evaluation logs (OPZIONALE)
EVAL_BASE = Path("/content/drive/MyDrive/OCR/evaluation")
EVAL_LOGS = EVAL_BASE / "logs"

# ==========================================
# FUNZIONI UTILITY
# ==========================================

def get_size_mb(path):
    """Calcola dimensione di file/directory in MB"""
    if not os.path.exists(path):
        return 0

    if os.path.isfile(path):
        return os.path.getsize(path) / (1024 * 1024)

    total = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if os.path.exists(fp):
                total += os.path.getsize(fp)
    return total / (1024 * 1024)

def remove_path(path, name):
    """Rimuove path con gestione errori"""
    if not os.path.exists(path):
        print(f"   ⏭️  {name}: già assente")
        return 0

    size = get_size_mb(path)

    try:
        if os.path.isfile(path):
            os.remove(path)
        else:
            shutil.rmtree(path)
        print(f"   ✅ {name}: rimosso ({size:.1f} MB)")
        return size
    except Exception as e:
        print(f"   ❌ {name}: errore - {e}")
        return 0

# ==========================================
# SCANSIONE INIZIALE
# ==========================================

print("\n📊 SCANSIONE SPAZIO OCCUPATO...")
print("-" * 70)

total_space = 0
items_to_clean = []

# 1. Database locali
print("\n1️⃣  Database Locali (/content):")
for path in LOCAL_TEMP_PATHS:
    size = get_size_mb(path)
    total_space += size
    status = "✅ Presente" if os.path.exists(path) else "⏭️  Assente"
    print(f"   {status}: {os.path.basename(path)} ({size:.1f} MB)")
    if os.path.exists(path):
        items_to_clean.append(('local', path, os.path.basename(path)))

# 2. Database Drive
print("\n2️⃣  Database Drive (permanenti):")
for path in DRIVE_DB_PATHS:
    size = get_size_mb(path)
    total_space += size
    status = "✅ Presente" if os.path.exists(path) else "⏭️  Assente"
    print(f"   {status}: {os.path.basename(path)} ({size:.1f} MB)")
    if os.path.exists(path):
        items_to_clean.append(('drive_db', path, os.path.basename(path)))

# 3. CLIP embeddings
print("\n3️⃣  CLIP Embeddings:")
size = get_size_mb(CLIP_JSON_PATH)
total_space += size
status = "✅ Presente" if os.path.exists(CLIP_JSON_PATH) else "⏭️  Assente"
print(f"   {status}: clip_embeddings.json ({size:.1f} MB)")
if os.path.exists(CLIP_JSON_PATH):
    items_to_clean.append(('clip', CLIP_JSON_PATH, 'clip_embeddings.json'))

# 4. OCR Output temporaneo
print("\n4️⃣  OCR Output Temporaneo:")
size = get_size_mb(OCR_OUTPUT_DIR)
total_space += size
status = "✅ Presente" if os.path.exists(OCR_OUTPUT_DIR) else "⏭️  Assente"
print(f"   {status}: OCR_Output_Temp ({size:.1f} MB)")
if os.path.exists(OCR_OUTPUT_DIR):
    items_to_clean.append(('ocr_temp', OCR_OUTPUT_DIR, 'OCR_Output_Temp'))

# 5. Evaluation logs
print("\n5️⃣  Evaluation Logs:")
size = get_size_mb(EVAL_LOGS)
total_space += size
status = "✅ Presente" if os.path.exists(EVAL_LOGS) else "⏭️  Assente"
print(f"   {status}: evaluation/logs ({size:.1f} MB)")
if os.path.exists(EVAL_LOGS):
    items_to_clean.append(('eval', EVAL_LOGS, 'evaluation/logs'))

print("\n" + "=" * 70)
print(f"📦 TOTALE SPAZIO OCCUPATO: {total_space:.1f} MB")
print("=" * 70)

# ==========================================
# CONFERMA UTENTE
# ==========================================

if not items_to_clean:
    print("\n✅ Nessun file da pulire. Tutto già pulito!")
else:
    print(f"\n⚠️  ATTENZIONE: Questa operazione rimuoverà {len(items_to_clean)} elementi.")
    print(f"   Spazio liberato: ~{total_space:.1f} MB")
    print("\n📋 Elementi da rimuovere:")

    # Raggruppa per tipo
    local_items = [x for x in items_to_clean if x[0] == 'local']
    drive_db_items = [x for x in items_to_clean if x[0] == 'drive_db']
    clip_items = [x for x in items_to_clean if x[0] == 'clip']
    ocr_items = [x for x in items_to_clean if x[0] == 'ocr_temp']
    eval_items = [x for x in items_to_clean if x[0] == 'eval']

    if local_items:
        print(f"\n   🗂️  Database Locali ({len(local_items)}):")
        for _, path, name in local_items:
            print(f"      • {name}")

    if drive_db_items:
        print(f"\n   💾 Database Drive ({len(drive_db_items)}):")
        for _, path, name in drive_db_items:
            print(f"      • {name}")

    if clip_items:
        print(f"\n   🖼️  CLIP Embeddings ({len(clip_items)}):")
        for _, path, name in clip_items:
            print(f"      • {name}")

    if ocr_items:
        print(f"\n   📄 OCR Output ({len(ocr_items)}):")
        for _, path, name in ocr_items:
            print(f"      • {name}")

    if eval_items:
        print(f"\n   📊 Evaluation Logs ({len(eval_items)}):")
        for _, path, name in eval_items:
            print(f"      • {name}")

    print("\n" + "=" * 70)
    print("🔴 QUESTA OPERAZIONE È IRREVERSIBILE!")
    print("=" * 70)

    # ==========================================
    # OPZIONI DI PULIZIA
    # ==========================================

    print("\n📝 Opzioni disponibili:")
    print("   1️⃣  PULIZIA COMPLETA (rimuove tutto)")
    print("   2️⃣  PULIZIA SELETTIVA (scegli cosa rimuovere)")
    print("   3️⃣  SOLO DATABASE (mantieni OCR output e logs)")
    print("   4️⃣  ANNULLA (non fare nulla)")

    choice = input("\n❓ Scegli un'opzione (1-4): ").strip()

    # ==========================================
    # ESECUZIONE PULIZIA
    # ==========================================

    freed_space = 0

    if choice == '1':
        # PULIZIA COMPLETA
        print("\n🚀 PULIZIA COMPLETA IN CORSO...")
        print("-" * 70)

        confirm = input("\n⚠️  Sei SICURO? Digita 'DELETE' per confermare: ").strip()

        if confirm == 'DELETE':
            for item_type, path, name in items_to_clean:
                freed_space += remove_path(path, name)

            print("\n" + "=" * 70)
            print(f"✅ PULIZIA COMPLETATA!")
            print(f"💾 Spazio liberato: {freed_space:.1f} MB")
            print("=" * 70)
        else:
            print("\n❌ Operazione annullata (conferma non corretta)")

    elif choice == '2':
        # PULIZIA SELETTIVA
        print("\n🎯 PULIZIA SELETTIVA")
        print("-" * 70)
        print("\nPer ogni elemento, scegli:")
        print("   • 's' = Sì, rimuovi")
        print("   • 'n' = No, mantieni")
        print()

        for item_type, path, name in items_to_clean:
            response = input(f"❓ Rimuovere '{name}'? (s/n): ").strip().lower()
            if response == 's':
                freed_space += remove_path(path, name)
            else:
                print(f"   ⏭️  {name}: mantenuto")

        print("\n" + "=" * 70)
        print(f"✅ PULIZIA SELETTIVA COMPLETATA!")
        print(f"💾 Spazio liberato: {freed_space:.1f} MB")
        print("=" * 70)

    elif choice == '3':
        # SOLO DATABASE
        print("\n🗄️  PULIZIA SOLO DATABASE")
        print("-" * 70)

        confirm = input("\n⚠️  Rimuovere tutti i database? Digita 'YES' per confermare: ").strip()

        if confirm == 'YES':
            # Rimuovi solo database e CLIP
            for item_type, path, name in items_to_clean:
                if item_type in ['local', 'drive_db', 'clip']:
                    freed_space += remove_path(path, name)

            print("\n" + "=" * 70)
            print(f"✅ DATABASE PULITI!")
            print(f"💾 Spazio liberato: {freed_space:.1f} MB")
            print(f"📄 OCR Output e Logs: mantenuti")
            print("=" * 70)
        else:
            print("\n❌ Operazione annullata (conferma non corretta)")

    elif choice == '4':
        print("\n✅ Operazione annullata dall'utente")

    else:
        print("\n❌ Opzione non valida. Operazione annullata.")

# ==========================================
# VERIFICA FINALE
# ==========================================

if 'freed_space' in locals() and freed_space > 0:
    print("\n🔍 VERIFICA FINALE...")
    print("-" * 70)

    remaining_items = []
    for item_type, path, name in items_to_clean:
        if os.path.exists(path):
            remaining_items.append(name)

    if remaining_items:
        print(f"\n⚠️  {len(remaining_items)} elementi ancora presenti:")
        for name in remaining_items:
            print(f"   • {name}")
    else:
        print("\n✅ Tutti gli elementi selezionati sono stati rimossi!")

    print("\n" + "=" * 70)
    print("📊 STATO FINALE:")
    print("=" * 70)
    print(f"   • Spazio liberato: {freed_space:.1f} MB")
    print(f"   • Database pronti per nuova ingestione: ✅")
    print("\n💡 Prossimi passi:")
    print("   1. Esegui BLOCCO 0 (Setup)")
    print("   2. Esegui BLOCCO 1 (Config) con nuovi parametri")
    print("   3. Esegui BLOCCO INGESTIONE")
    print("=" * 70)

print("\n🎉 Script completato!") '''

#cella di ingestion

In [ ]:
''' # ==========================================
# BLOCCO INGESTIONE: DOLPHIN OCR + DATABASE CREATION
# ==========================================

import os
import glob
import shutil
import gc
import torch
import subprocess
import re
import json
from urllib.parse import unquote
from pathlib import Path
from PIL import Image
import time
import logging
import warnings
from tqdm.notebook import tqdm
import fitz  # PyMuPDF per batch splitting

# Sopprimi warning
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("🚀 BLOCCO INGESTIONE: Dolphin OCR + Database Creation")
print("="*70)

# ==========================================
# VERIFICA BLOCCO 1 (Config) ESEGUITO
# ==========================================

print("\n⚙️ Verifica configurazione...")

try:
    # Verifica che le variabili dal BLOCCO 1 esistano
    _required_vars = [
        ('PDF_INPUT_DIR', PDF_INPUT_DIR),
        ('OCR_OUTPUT_DIR', OCR_OUTPUT_DIR),
        ('LOCAL_TEXT_DB', LOCAL_TEXT_DB),
        ('LOCAL_IMAGE_DB', LOCAL_IMAGE_DB),
        ('DRIVE_TEXT_DB', DRIVE_TEXT_DB),
        ('DRIVE_IMAGE_DB', DRIVE_IMAGE_DB),
        ('DRIVE_CLIP_JSON', DRIVE_CLIP_JSON),
        ('TEXT_EMBED_MODEL', TEXT_EMBED_MODEL),
        ('CLIP_MODEL_NAME', CLIP_MODEL_NAME),
        ('TEXT_COLLECTION_NAME', TEXT_COLLECTION_NAME),
        ('IMAGE_COLLECTION_NAME', IMAGE_COLLECTION_NAME),
        ('PAGES_PER_BATCH', PAGES_PER_BATCH),
        ('DOLPHIN_TIMEOUT', DOLPHIN_TIMEOUT),
        ('CHUNK_SIZE', CHUNK_SIZE),
        ('CHUNK_OVERLAP', CHUNK_OVERLAP),
        ('CLIP_SIZE_KB_MIN', CLIP_SIZE_KB_MIN),
        ('CLIP_WIDTH_MIN', CLIP_WIDTH_MIN),
        ('CLIP_HEIGHT_MIN', CLIP_HEIGHT_MIN),
        ('CLIP_CAPTION_MAX_LEN', CLIP_CAPTION_MAX_LEN),
    ]

    print("✅ Configurazione da BLOCCO 1 trovata")

except NameError as e:
    print("\n" + "="*70)
    print("❌ ERRORE: BLOCCO 1 (Config) non è stato eseguito!")
    print("="*70)
    print(f"\nVariabile mancante: {e}")
    print("\n🔧 SOLUZIONE:")
    print("   1. Torna indietro")
    print("   2. Esegui il BLOCCO 1 (Config)")
    print("   3. Poi riesegui questo blocco\n")
    print("="*70)
    raise

# ==========================================
# VERIFICA ESISTENZA DATABASE
# ==========================================

print("\n📊 Verifica database esistenti...")

text_db_exists = os.path.exists(DRIVE_TEXT_DB)
img_db_exists = os.path.exists(DRIVE_IMAGE_DB)
clip_json_exists = os.path.exists(DRIVE_CLIP_JSON)

print(f"   {'✅' if text_db_exists else '❌'} Text DB: {DRIVE_TEXT_DB}")
print(f"   {'✅' if img_db_exists else '❌'} Image DB: {DRIVE_IMAGE_DB}")
print(f"   {'✅' if clip_json_exists else '❌'} CLIP JSON: {DRIVE_CLIP_JSON}")

if text_db_exists and img_db_exists:
    print("\n⚠️ ATTENZIONE: Database già esistenti!")
    print("   Se continui, i database esistenti verranno SOVRASCRITTI.")
    print("   Questo processo richiede 30-60 minuti.")

    response = input("\n❓ Vuoi continuare e sovrascrivere? (yes/no): ")

    if response.lower() not in ['yes', 'y']:
        print("\n✅ Operazione annullata. Database esistenti preservati.")
        print("   Puoi procedere con i BLOCCHI 2, 3 o 4.")
        raise SystemExit("User cancelled operation")
    else:
        print("\n⚠️ Rimozione database esistenti...")
        if os.path.exists(DRIVE_TEXT_DB):
            shutil.rmtree(DRIVE_TEXT_DB)
            print(f"   🗑️ Rimosso: {DRIVE_TEXT_DB}")
        if os.path.exists(DRIVE_IMAGE_DB):
            shutil.rmtree(DRIVE_IMAGE_DB)
            print(f"   🗑️ Rimosso: {DRIVE_IMAGE_DB}")
        if os.path.exists(DRIVE_CLIP_JSON):
            os.remove(DRIVE_CLIP_JSON)
            print(f"   🗑️ Rimosso: {DRIVE_CLIP_JSON}")

# ==========================================
# PRINT CONFIGURAZIONE
# ==========================================

print(f"""
{"="*70}
⚙️ CONFIGURAZIONE INGESTIONE:
{"="*70}

📂 Input/Output:
   • PDF Input: {PDF_INPUT_DIR}
   • OCR Output: {OCR_OUTPUT_DIR}
   • Drive Text DB: {DRIVE_TEXT_DB}
   • Drive Image DB: {DRIVE_IMAGE_DB}
   • CLIP JSON: {DRIVE_CLIP_JSON}

🤖 Modelli:
   • Text Embed: {TEXT_EMBED_MODEL}
   • CLIP: {CLIP_MODEL_NAME}

🐬 Dolphin Batch:
   • Pagine/batch: {PAGES_PER_BATCH}
   • Timeout: {DOLPHIN_TIMEOUT}s

📝 Text Chunking:
   • Chunk size: {CHUNK_SIZE}
   • Overlap: {CHUNK_OVERLAP}

🖼️ CLIP Smart Threshold:
   • Min size: {CLIP_SIZE_KB_MIN} KB
   • Min width: {CLIP_WIDTH_MIN} px
   • Min height: {CLIP_HEIGHT_MIN} px
   • Caption max: {CLIP_CAPTION_MAX_LEN} char

{"="*70}
""")

# ==========================================
# UTILITY FUNCTIONS
# ==========================================

def split_pdf_in_batches(pdf_path, output_dir, pages_per_batch):
    """
    Divide PDF in batch più piccoli per Dolphin
    """
    doc = fitz.open(pdf_path)
    total_pages = len(doc)

    batch_paths = []
    os.makedirs(output_dir, exist_ok=True)

    for start in range(0, total_pages, pages_per_batch):
        end = min(start + pages_per_batch, total_pages)
        batch_name = f"batch_{start:04d}_{end:04d}.pdf"
        batch_path = os.path.join(output_dir, batch_name)

        # Crea PDF batch
        batch_doc = fitz.open()
        batch_doc.insert_pdf(doc, from_page=start, to_page=end-1)
        batch_doc.save(batch_path)
        batch_doc.close()

        batch_paths.append((batch_path, start, end))

    doc.close()
    return batch_paths, total_pages

def run_dolphin_on_batch(batch_pdf, output_dir, timeout):
    """
    Esegue Dolphin su un singolo batch
    """
    cmd = f"""
    export TF_CPP_MIN_LOG_LEVEL=3
    export CUDA_VISIBLE_DEVICES=0
    timeout {timeout} python /content/Dolphin/demo_page.py \
        --model_path /content/hf_model \
        --input_path "{batch_pdf}" \
        --save_dir "{output_dir}" 2>&1
    """

    try:
        result = subprocess.run(
            cmd,
            shell=True,
            capture_output=True,
            text=True,
            timeout=timeout + 10
        )

        # Verifica successo
        md_files = glob.glob(os.path.join(output_dir, "markdown", "*.md"))
        if md_files and result.returncode != 124:
            return True, len(md_files)
        return False, 0
    except Exception as e:
        return False, 0

def merge_batch_results(manual_name, batch_results, final_output_dir):
    """
    Unisce i risultati dei batch in un unico output
    """
    os.makedirs(final_output_dir, exist_ok=True)
    final_md_dir = os.path.join(final_output_dir, "markdown")
    os.makedirs(final_md_dir, exist_ok=True)

    all_md_content = []
    page_counter = 1

    # Merge markdown files
    for batch_dir, start_page, end_page in sorted(batch_results):
        batch_md_dir = os.path.join(batch_dir, "markdown")
        if os.path.exists(batch_md_dir):
            md_files = sorted(glob.glob(os.path.join(batch_md_dir, "*.md")))

            for md_file in md_files:
                with open(md_file, 'r', encoding='utf-8') as f:
                    content = f.read()
                    all_md_content.append(f"## Pagina {page_counter}\n\n{content}\n")
                    page_counter += 1

        # Copia immagini
        batch_images = glob.glob(os.path.join(batch_dir, "*.png"))
        for img in batch_images:
            shutil.copy(img, final_output_dir)

    # Salva markdown unificato
    unified_md = os.path.join(final_md_dir, f"{manual_name}.md")
    with open(unified_md, 'w', encoding='utf-8') as f:
        f.write("\n".join(all_md_content))

    # Pulizia batch temporanei
    for batch_dir, _, _ in batch_results:
        if os.path.exists(batch_dir):
            shutil.rmtree(batch_dir)

def save_to_drive(local_path, drive_path):
    """Salva su Drive"""
    print(f"💾 Salvataggio: {os.path.basename(drive_path)}...")
    if os.path.exists(drive_path):
        shutil.rmtree(drive_path)
    shutil.copytree(local_path, drive_path)
    print("✅ Salvato.")

def should_use_clip(img_path, caption):
    """
    Smart threshold per decidere se usare CLIP
    """
    try:
        # Check dimensione file
        size_kb = os.path.getsize(img_path) / 1024
        if size_kb < CLIP_SIZE_KB_MIN:
            return False

        # Check dimensioni immagine
        with Image.open(img_path) as img:
            width, height = img.size
            if width < CLIP_WIDTH_MIN or height < CLIP_HEIGHT_MIN:
                return False

        # Check caption
        caption_short = len(caption.strip()) < CLIP_CAPTION_MAX_LEN
        caption_generic = caption.lower().startswith(("figura", "fig.", "immagine", "image"))

        # Usa CLIP se: grande + (caption corta O generica)
        return caption_short or caption_generic

    except:
        return False

# ==========================================
# FASE 1: DOLPHIN OCR BATCH
# ==========================================

def run_batch_dolphin_ocr():
    """
    Processa PDF con Dolphin in batch
    """
    print("\n" + "="*70)
    print("🐬 FASE 1: DOLPHIN OCR BATCH PROCESSING")
    print("="*70)

    if not os.path.exists(PDF_INPUT_DIR):
        print(f"❌ Cartella PDF non trovata: {PDF_INPUT_DIR}")
        return False

    pdf_files = glob.glob(f"{PDF_INPUT_DIR}/*.pdf")
    if not pdf_files:
        print("❌ Nessun PDF trovato")
        return False

    print(f"\n📄 Trovati {len(pdf_files)} PDF da processare")
    os.makedirs(OCR_OUTPUT_DIR, exist_ok=True)

    pbar_pdf = tqdm(pdf_files, desc="🔄 OCR Batch Dolphin", unit="PDF")

    for pdf in pbar_pdf:
        name = Path(pdf).stem
        final_output = os.path.join(OCR_OUTPUT_DIR, name)

        pbar_pdf.set_description(f"🔄 {name[:20]}...")

        # Skip se già processato
        if os.path.exists(final_output):
            md_files = glob.glob(os.path.join(final_output, "markdown", "*.md"))
            if md_files:
                pbar_pdf.write(f"⏭️ Skip: {name} (già processato)")
                continue

        pdf_size_mb = os.path.getsize(pdf) / (1024 * 1024)
        pbar_pdf.write(f"\n📋 {name} ({pdf_size_mb:.1f} MB)")

        # Split in batches
        temp_batch_dir = os.path.join(OCR_OUTPUT_DIR, f"{name}_batches")
        batch_paths, total_pages = split_pdf_in_batches(pdf, temp_batch_dir, PAGES_PER_BATCH)

        num_batches = len(batch_paths)
        pbar_pdf.write(f"   📦 Diviso in {num_batches} batch ({PAGES_PER_BATCH} pagine/batch)")

        # Processa ogni batch
        batch_results = []
        pbar_batch = tqdm(batch_paths, desc="   🐬 Batch", leave=False, unit="batch")

        for batch_pdf, start, end in pbar_batch:
            batch_name = f"pg{start}-{end}"
            batch_output = os.path.join(temp_batch_dir, f"output_{batch_name}")

            pbar_batch.set_description(f"   🐬 Pagine {start}-{end}")

            start_time = time.time()
            success, num_pages = run_dolphin_on_batch(batch_pdf, batch_output, DOLPHIN_TIMEOUT)
            elapsed = time.time() - start_time

            if success:
                pbar_pdf.write(f"      ✅ Batch {batch_name}: {num_pages} pagine ({elapsed/60:.1f} min)")
                batch_results.append((batch_output, start, end))
            else:
                pbar_pdf.write(f"      ⚠️ Batch {batch_name}: fallito/timeout")

        pbar_batch.close()

        # Merge risultati
        if batch_results:
            pbar_pdf.write(f"   🔗 Merge {len(batch_results)} batch...")
            merge_batch_results(name, batch_results, final_output)
            pbar_pdf.write(f"   ✅ {name} completato")
        else:
            pbar_pdf.write(f"   ❌ {name} fallito completamente")

        # Cleanup batch directory
        if os.path.exists(temp_batch_dir):
            shutil.rmtree(temp_batch_dir)

    pbar_pdf.close()

    print("\n✅ FASE 1 COMPLETATA: Dolphin OCR")
    return True

# ==========================================
# FASE 2: TEXT DATABASE
# ==========================================

def create_text_db():
    """Crea database testuale"""
    print("\n" + "="*70)
    print("📚 FASE 2: CREAZIONE TEXT DATABASE")
    print("="*70)

    from langchain_community.document_loaders import TextLoader
    from langchain_text_splitters import RecursiveCharacterTextSplitter
    from langchain_huggingface import HuggingFaceEmbeddings
    try:
        from langchain_chroma import Chroma
    except ImportError:
        from langchain_community.vectorstores import Chroma
    from langchain_core.documents import Document

    md_files = glob.glob(f"{OCR_OUTPUT_DIR}/**/*.md", recursive=True)

    if not md_files:
        print("❌ Nessun file markdown trovato!")
        return False

    docs = []
    for md in tqdm(md_files, desc="📄 Caricamento MD", unit="file"):
        try:
            loader = TextLoader(md, encoding='utf-8')
            loaded_docs = loader.load()
            manual_name = Path(md).parents[1].name

            for doc in loaded_docs:
                doc.metadata['manual'] = manual_name
                doc.metadata['source'] = md
                doc.metadata['language'] = 'it'

            docs.extend(loaded_docs)
        except:
            pass

    if not docs:
        print("❌ Nessun documento caricato")
        return False

    print(f"   📝 Chunking {len(docs)} documenti...")
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    chunks = splitter.split_documents(docs)
    print(f"   ✅ {len(chunks)} chunks creati")

    print("   🔧 Caricamento Embedding...")
    embed_model = HuggingFaceEmbeddings(
        model_name=TEXT_EMBED_MODEL,
        model_kwargs={'device': 'cuda'},
        encode_kwargs={'normalize_embeddings': True}
    )

    if os.path.exists(LOCAL_TEXT_DB):
        shutil.rmtree(LOCAL_TEXT_DB)
        time.sleep(2)
    os.makedirs(LOCAL_TEXT_DB, exist_ok=True)

    print("   ⚡ Creazione vector store...")
    Chroma.from_documents(
        chunks,
        embed_model,
        persist_directory=LOCAL_TEXT_DB,
        collection_name=TEXT_COLLECTION_NAME,
        collection_metadata={"hnsw:space": "cosine"}
    )
    print(f"✅ Text DB creato ({len(chunks)} chunks)")

    save_to_drive(LOCAL_TEXT_DB, DRIVE_TEXT_DB)

    del embed_model
    gc.collect()
    torch.cuda.empty_cache()

    print("\n✅ FASE 2 COMPLETATA: Text Database")
    return True

# ==========================================
# FASE 3: IMAGE DATABASE + CLIP
# ==========================================

def create_image_db_with_selective_clip():
    """
    Crea DB immagini con embedding testuale + CLIP selettivo
    """
    print("\n" + "="*70)
    print("🖼️ FASE 3: CREAZIONE IMAGE DATABASE + CLIP SELETTIVO")
    print("="*70)

    from langchain_community.document_loaders import TextLoader
    from langchain_huggingface import HuggingFaceEmbeddings
    try:
        from langchain_chroma import Chroma
    except ImportError:
        from langchain_community.vectorstores import Chroma
    from langchain_core.documents import Document
    from transformers import CLIPModel, CLIPProcessor

    # Trova tutte le immagini
    all_pngs = glob.glob(f"{OCR_OUTPUT_DIR}/**/*.png", recursive=True)
    all_jpgs = glob.glob(f"{OCR_OUTPUT_DIR}/**/*.jpg", recursive=True)
    all_images = all_pngs + all_jpgs

    if not all_images:
        print("❌ Nessuna immagine trovata")
        return False

    img_map = {os.path.basename(p): p for p in all_images}
    print(f"   ✅ Mappate {len(img_map)} immagini")

    # Estrai metadata immagini dai markdown
    all_images_data = []
    md_files = glob.glob(f"{OCR_OUTPUT_DIR}/**/*.md", recursive=True)

    for md_file in md_files:
        manual_name = Path(md_file).parents[1].name
        with open(md_file, 'r', encoding='utf-8') as f:
            content = f.read()

        images_found = re.findall(r'!\s*\[(.*?)\]\s*\((.*?\.(?:png|jpg|jpeg))\)', content, re.IGNORECASE)

        count = 0
        for caption, rel_path in images_found:
            clean_filename = unquote(os.path.basename(rel_path))

            if clean_filename in img_map:
                full_img_path = img_map[clean_filename]
                idx = content.find(rel_path)
                start = max(0, idx - 200)
                end = min(len(content), idx + 200)
                context = content[start:end].replace("\n", " ").strip()

                all_images_data.append({
                    "path": full_img_path,
                    "manual": manual_name,
                    "caption": caption,
                    "context": context
                })
                count += 1

        print(f"   📄 {manual_name}: {count} immagini")

    if not all_images_data:
        print("❌ Nessuna immagine da indicizzare")
        return False

    # Fase 1: Crea DB con embedding testuale per TUTTE le immagini
    print(f"\n   🔧 Caricamento Text Embedding...")
    text_embed = HuggingFaceEmbeddings(
        model_name=TEXT_EMBED_MODEL,
        model_kwargs={'device': 'cuda'},
        encode_kwargs={'normalize_embeddings': True}
    )

    docs = []
    clip_candidates = []  # Immagini che avranno CLIP

    for i, img_data in enumerate(tqdm(all_images_data, desc="   📊 Analisi immagini", unit="img")):
        text_content = f"{img_data['caption']}. {img_data['context']}"

        # Determina se usare CLIP
        use_clip = should_use_clip(img_data['path'], img_data['caption'])

        doc = Document(
            page_content=text_content,
            metadata={
                "image_path": img_data['path'],
                "manual": img_data['manual'],
                "caption": img_data['caption'],
                "type": "image",
                "has_clip": use_clip,  # Flag per indicare CLIP disponibile
                "image_id": f"img_{i}"
            }
        )
        docs.append(doc)

        if use_clip:
            clip_candidates.append((f"img_{i}", img_data['path']))

    print(f"   ✅ {len(docs)} immagini totali")
    print(f"   🎯 {len(clip_candidates)} candidate per CLIP ({len(clip_candidates)/len(docs)*100:.1f}%)")

    # Crea DB con embedding testuale
    if os.path.exists(LOCAL_IMAGE_DB):
        shutil.rmtree(LOCAL_IMAGE_DB)
        time.sleep(2)
    os.makedirs(LOCAL_IMAGE_DB, exist_ok=True)

    print("   ⚡ Creazione vector store (testuale)...")
    Chroma.from_documents(
        docs,
        text_embed,
        persist_directory=LOCAL_IMAGE_DB,
        collection_name=IMAGE_COLLECTION_NAME,
        collection_metadata={"hnsw:space": "cosine"}
    )
    print(f"✅ Image DB creato (testuale)")

    del text_embed
    gc.collect()
    torch.cuda.empty_cache()

    # Fase 2: Pre-calcola CLIP embeddings per candidati selezionati
    if clip_candidates:
        print(f"\n   🔧 Caricamento CLIP per {len(clip_candidates)} immagini selezionate...")

        clip_model = CLIPModel.from_pretrained(CLIP_MODEL_NAME)
        clip_processor = CLIPProcessor.from_pretrained(CLIP_MODEL_NAME)
        clip_model = clip_model.to('cuda')
        clip_model.eval()

        clip_embeddings = {}
        batch_size = 8

        pbar = tqdm(range(0, len(clip_candidates), batch_size),
                    desc="   ⚡ CLIP Embedding", unit="batch")

        for i in pbar:
            batch = clip_candidates[i:i+batch_size]
            batch_images = []
            batch_ids = []

            for img_id, img_path in batch:
                try:
                    img = Image.open(img_path).convert('RGB')
                    if max(img.size) > 1024:
                        img.thumbnail((1024, 1024), Image.Resampling.LANCZOS)
                    batch_images.append(img)
                    batch_ids.append(img_id)
                except:
                    continue

            if batch_images:
                inputs = clip_processor(images=batch_images, return_tensors="pt", padding=True)
                inputs = {k: v.to('cuda') for k, v in inputs.items()}

                with torch.no_grad():
                    image_features = clip_model.get_image_features(**inputs)
                    image_features = image_features / image_features.norm(dim=-1, keepdim=True)

                for j, img_id in enumerate(batch_ids):
                    clip_embeddings[img_id] = image_features[j].cpu().numpy().tolist()

            pbar.set_postfix({"embeddings": len(clip_embeddings)})

        pbar.close()
        print(f"   ✅ {len(clip_embeddings)} CLIP embeddings generati")

        # Salva CLIP embeddings in JSON separato
        clip_json_path = DRIVE_CLIP_JSON
        os.makedirs(os.path.dirname(clip_json_path), exist_ok=True)
        with open(clip_json_path, 'w') as f:
            json.dump(clip_embeddings, f)
        print(f"   💾 CLIP embeddings salvati: {clip_json_path}")

        del clip_model, clip_processor
        gc.collect()
        torch.cuda.empty_cache()

    save_to_drive(LOCAL_IMAGE_DB, DRIVE_IMAGE_DB)

    print("\n✅ FASE 3 COMPLETATA: Image Database + CLIP")
    return True

# ==========================================
# ESECUZIONE PRINCIPALE
# ==========================================

print("\n" + "="*70)
print("🚀 INIZIO PROCESSO INGESTIONE")
print("="*70)

start_total = time.time()

try:
    # Fase 1: Dolphin OCR
    dolphin_success = run_batch_dolphin_ocr()

    if not dolphin_success:
        raise RuntimeError("Dolphin OCR fallito")

    # Fase 2: Text DB
    text_db_success = create_text_db()

    if not text_db_success:
        raise RuntimeError("Text DB creation fallito")

    # Fase 3: Image DB + CLIP
    image_db_success = create_image_db_with_selective_clip()

    if not image_db_success:
        raise RuntimeError("Image DB creation fallito")

    elapsed_total = time.time() - start_total

    print("\n" + "="*70)
    print("🎉 PROCESSO INGESTIONE COMPLETATO!")
    print("="*70)
    print(f"\n⏱️ Tempo totale: {elapsed_total/60:.1f} minuti")
    print(f"\n📊 Output:")
    print(f"   • Text DB: {DRIVE_TEXT_DB}")
    print(f"   • Image DB: {DRIVE_IMAGE_DB}")
    print(f"   • CLIP JSON: {DRIVE_CLIP_JSON}")

    # Verifica finale
    print(f"\n🔍 Verifica finale:")
    if os.path.exists(DRIVE_TEXT_DB):
        print(f"   ✅ Text DB creato correttamente")
    else:
        print(f"   ❌ Text DB NON trovato!")

    if os.path.exists(DRIVE_IMAGE_DB):
        print(f"   ✅ Image DB creato correttamente")
    else:
        print(f"   ❌ Image DB NON trovato!")

    if os.path.exists(DRIVE_CLIP_JSON):
        clip_size = os.path.getsize(DRIVE_CLIP_JSON) / (1024 * 1024)
        print(f"   ✅ CLIP JSON creato correttamente ({clip_size:.2f} MB)")
    else:
        print(f"   ⚠️ CLIP JSON NON trovato (normale se nessuna immagine qualificata)")

    print("\n✅ Ora puoi procedere con i BLOCCHI 2, 3 o 4!")

except Exception as e:
    print(f"\n❌ ERRORE durante ingestione: {str(e)}")
    import traceback
    traceback.print_exc()

    print("\n🔧 Possibili soluzioni:")
    print("   1. Verifica che Dolphin sia installato (BLOCCO 0)")
    print("   2. Verifica connessione GPU")
    print("   3. Verifica spazio su Drive (>5GB liberi)")
    print("   4. Controlla log errori sopra")

print("\n" + "="*70) '''

#Evaluation Framework

In [ ]:
# ==========================================
# BLOCCO 2: EVALUATION & TESTING FRAMEWORK (v3.0 - AUTO-UPDATE REPORT)
# ==========================================

import os
import json
import re
import numpy as np
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Tuple, Optional
import logging
from dataclasses import dataclass, asdict

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("🚀 BLOCCO 2: Evaluation & Testing Framework (v3.0 - AUTO-UPDATE)")
print("=" * 70)

# ==========================================
# SETUP DIRECTORIES SU DRIVE
# ==========================================

class PathConfig:
    """Configurazione path persistenti su Drive"""

    DRIVE_BASE = Path("/content/drive/MyDrive/OCR")
    EVALUATION_BASE = DRIVE_BASE / "evaluation"

    # Subdirectories
    LOGS_DIR = EVALUATION_BASE / "logs"
    REPORTS_DIR = EVALUATION_BASE / "reports"
    INJECTION_TESTS_DIR = EVALUATION_BASE / "injection_tests"

    # Files
    QUERIES_LOG = LOGS_DIR / "queries.jsonl"
    SUMMARY_LOG = LOGS_DIR / "metrics_summary.json"
    INJECTION_RESULTS = INJECTION_TESTS_DIR / "injection_results.json"
    EVALUATION_REPORT = REPORTS_DIR / "automate_evaluation_report.md"

    @classmethod
    def setup_directories(cls):
        """Crea tutte le directory necessarie"""
        for dir_path in [cls.LOGS_DIR, cls.REPORTS_DIR, cls.INJECTION_TESTS_DIR]:
            dir_path.mkdir(parents=True, exist_ok=True)
        print(f"✅ Directory create in: {cls.EVALUATION_BASE}")

# Setup iniziale
PathConfig.setup_directories()

print(f"""
📁 Struttura creata:
   {PathConfig.EVALUATION_BASE}/
   ├── logs/
   │   ├── queries.jsonl
   │   └── metrics_summary.json
   ├── reports/
   │   └── automate_evaluation_report.md (auto-updated)
   └── injection_tests/
       └── injection_results.json
""")

# ==========================================
# 1. CONFIDENCE CALCULATOR
# ==========================================

class ConfidenceCalculator:
    """
    Calcola confidence da 3 fonti indipendenti
    Reference-free approach (no ground truth needed)
    """

    @staticmethod
    def retrieval_confidence(docs_with_scores: List[Tuple]) -> float:
        """
        Score basato su similarity e consistency dei retrieved chunks

        Args:
            docs_with_scores: List of (Document, distance_score) tuples

        Returns:
            float: Confidence score [0, 1]
        """
        if not docs_with_scores:
            return 0.0

        # Convert distance to similarity (Chroma usa L2 distance)
        scores = [1 - min(score, 1.0) for _, score in docs_with_scores]

        # Componente 1: Similarity media
        avg_similarity = np.mean(scores)

        # Componente 2: Consistency (bassa varianza = alta consistency)
        consistency = 1 - np.std(scores) if len(scores) > 1 else 1.0

        # Componente 3: Penalità per troppa diversità di manuali
        manuals = set(doc.metadata.get('manual', 'unknown') for doc, _ in docs_with_scores)
        diversity_penalty = 0.95 if len(manuals) > 2 else 1.0

        # Weighted combination
        confidence = (avg_similarity * 0.6 + consistency * 0.4) * diversity_penalty

        return round(confidence, 3)

    @staticmethod
    def context_relevance(query: str, chunks: List, embed_model) -> float:
        """
        Semantic similarity tra query e retrieved context

        Args:
            query: User query string
            chunks: List of Document objects
            embed_model: HuggingFaceEmbeddings model

        Returns:
            float: Relevance score [0, 1]
        """
        if not chunks:
            return 0.0

        try:
            # Query embedding
            query_emb = embed_model.embed_query(query)
            query_emb = np.array(query_emb)

            relevances = []
            for chunk in chunks:
                # Chunk embedding (primi 500 char per efficienza)
                chunk_text = chunk.page_content[:500]
                chunk_emb = embed_model.embed_query(chunk_text)
                chunk_emb = np.array(chunk_emb)

                # Cosine similarity
                similarity = np.dot(query_emb, chunk_emb) / (
                    np.linalg.norm(query_emb) * np.linalg.norm(chunk_emb)
                )
                relevances.append(similarity)

            return round(np.mean(relevances), 3)

        except Exception as e:
            logger.warning(f"Context relevance calculation failed: {e}")
            return 0.5  # Neutral fallback

    @staticmethod
    def answer_quality(response: str) -> float:
        """
        Heuristics per valutare qualità della risposta

        Args:
            response: Generated answer string

        Returns:
            float: Quality score [0, 1]
        """
        if not response or len(response.strip()) < 10:
            return 0.0

        score = 0.0
        words = response.split()
        length = len(words)

        # 1. Lunghezza ottimale (35% peso)
        if 50 < length < 300:
            score += 0.35
        elif 30 < length <= 50:
            score += 0.20
        elif length >= 300:
            score += 0.15

        # 2. Dettagli tecnici (25% peso)
        # Numeri con unità di misura
        if re.search(r'\d+\s*(km|m|cm|mm|kg|g|°C|bar|V|A|kW|rpm|l|ml)', response):
            score += 0.15

        # Keywords automotive
        automotive_keywords = ['premere', 'ruotare', 'spegnere', 'accendere',
                               'verificare', 'controllare', 'impostare', 'selezionare',
                               'mantenere', 'rilasciare', 'press', 'turn', 'switch']
        if any(kw in response.lower() for kw in automotive_keywords):
            score += 0.10

        # 3. Struttura articolata (20% peso)
        sentences = [s.strip() for s in response.split('.') if len(s.strip()) > 10]
        if len(sentences) >= 3:
            score += 0.15
        elif len(sentences) == 2:
            score += 0.10
        else:
            score += 0.05

        # 4. Markers di procedura (10% peso)
        procedure_markers = ['primo', 'secondo', 'infine', 'successivamente',
                             'quindi', 'passo', '1.', '2.', 'a)', 'b)', 'first', 'second']
        if any(marker in response.lower() for marker in procedure_markers):
            score += 0.10

        # 5. Non è risposta di fallback (10% peso)
        fallback_phrases = ['non riesco', 'non posso', 'non ho informazioni',
                            'non è presente', 'mi scuso', 'cannot', 'unable']
        if not any(phrase in response.lower() for phrase in fallback_phrases):
            score += 0.10

        return round(min(score, 1.0), 3)

    @staticmethod
    def aggregate(retrieval: float, relevance: float, quality: float) -> Dict:
        """
        Combina i 3 score in confidence finale

        Args:
            retrieval: Retrieval confidence [0, 1]
            relevance: Context relevance [0, 1]
            quality: Answer quality [0, 1]

        Returns:
            dict: {score, label, color, breakdown}
        """
        # Weighted average (pesi ottimizzati per RAG)
        weights = {
            'retrieval': 0.40,   # Qualità chunks retrieved
            'relevance': 0.35,   # Rilevanza semantica
            'quality': 0.25      # Qualità risposta
        }

        final = (
            retrieval * weights['retrieval'] +
            relevance * weights['relevance'] +
            quality * weights['quality']
        )

        # Classificazione per UI
        if final >= 0.75:
            label = "HIGH"
            color = "#4CAF50"  # Green
        elif final >= 0.55:
            label = "MEDIUM"
            color = "#FF9800"  # Orange
        else:
            label = "LOW"
            color = "#F44336"  # Red

        return {
            'score': round(final, 3),
            'label': label,
            'color': color,
            'breakdown': {
                'retrieval': round(retrieval, 3),
                'relevance': round(relevance, 3),
                'quality': round(quality, 3)
            }
        }

print("✅ ConfidenceCalculator loaded")

# ==========================================
# 2. LLM-AS-JUDGE
# ==========================================

class LLMJudge:
    """
    Usa Gemini come arbitro per valutazione qualitativa
    """

    def __init__(self, llm):
        """
        Args:
            llm: ChatGoogleGenerativeAI instance
        """
        self.llm = llm

    def evaluate_response(self, query: str, context: str, response: str) -> Dict:
        """
        Valuta risposta su 3 dimensioni: Faithfulness, Relevance, Completeness

        Args:
            query: User query
            context: Retrieved context (concatenated chunks)
            response: Generated answer

        Returns:
            dict: {faithfulness, relevance, completeness, average, reasoning}
        """

        # Truncate context se troppo lungo (Gemini limit)
        max_context_chars = 2000
        if len(context) > max_context_chars:
            context = context[:max_context_chars] + "\n... [truncated]"

        judge_prompt = f"""Evaluate this RAG response (scale 1-5):

CONTEXT (from manual):
{context}

USER QUESTION:
{query}

GENERATED ANSWER:
{response}

EVALUATE according to these 3 criteria (scale 1-5):

1. **FAITHFULNESS** (Fidelity to context)
   - 1: Invented or contradictory information
   - 3: Partially supported by context
   - 5: Completely faithful to context

2. **RELEVANCE** (Pertinence to question)
   - 1: Does not answer the question
   - 3: Partial or tangential answer
   - 5: Direct and complete answer

3. **COMPLETENESS** (Technical detail)
   - 1: Vague, missing essential details
   - 3: Sufficient but not detailed
   - 5: Very detailed with technical specs

IMPORTANT: Respond ONLY with valid JSON (NO markdown, NO extra explanations):
{{
  "faithfulness": <number 1-5>,
  "relevance": <number 1-5>,
  "completeness": <number 1-5>,
  "reasoning": "<max 100 characters>"
}}"""

        try:
            # Invoke LLM
            judge_response = self.llm.invoke(judge_prompt)

            # Extract content from AIMessage object
            if hasattr(judge_response, 'content'):
                json_str = judge_response.content
            else:
                json_str = str(judge_response)

            json_str = json_str.strip()

            # Rimuovi markdown wrapper se presente
            json_str = re.sub(r'```json\s*', '', json_str)
            json_str = re.sub(r'```\s*$', '', json_str)
            json_str = re.sub(r'^```\s*', '', json_str)

            # Parse
            scores = json.loads(json_str)

            # Validazione
            required_keys = ['faithfulness', 'relevance', 'completeness']
            for key in required_keys:
                if key not in scores:
                    raise ValueError(f"Missing key: {key}")
                if not isinstance(scores[key], (int, float)):
                    raise ValueError(f"Invalid type for {key}")
                if not (1 <= scores[key] <= 5):
                    raise ValueError(f"Score out of range for {key}: {scores[key]}")

            # Calcola average
            scores['average'] = round(
                (scores['faithfulness'] + scores['relevance'] + scores['completeness']) / 3,
                2
            )

            # Assicura reasoning sia stringa
            if 'reasoning' not in scores:
                scores['reasoning'] = "No reasoning provided"

            logger.info(f"LLM Judge completed: avg={scores['average']}")
            return scores

        except json.JSONDecodeError as e:
            logger.error(f"JSON parse failed: {e}")
            raw_preview = json_str[:200] if 'json_str' in locals() else "N/A"
            logger.error(f"Raw response preview: {raw_preview}")
            return {
                'faithfulness': 0,
                'relevance': 0,
                'completeness': 0,
                'average': 0,
                'reasoning': f"JSON parse error: {str(e)[:50]}"
            }
        except Exception as e:
            logger.error(f"LLM Judge failed: {e}")
            return {
                'faithfulness': 0,
                'relevance': 0,
                'completeness': 0,
                'average': 0,
                'reasoning': f"Error: {str(e)[:50]}"
            }

print("✅ LLMJudge loaded")

# ==========================================
# 3. RETRIEVAL METRICS
# ==========================================

class RetrievalMetrics:
    """
    Metriche intrinseche di retrieval (no ground truth)
    """

    @staticmethod
    def diversity_score(retrieved_docs: List) -> float:
        """
        Diversità manuali: quanti manuali diversi nei retrieved chunks?

        Returns:
            float: Ratio unique_manuals / total_docs [0, 1]
        """
        if not retrieved_docs:
            return 0.0

        manuals = [doc.metadata.get('manual', 'unknown') for doc in retrieved_docs]
        unique_manuals = len(set(manuals))

        return round(unique_manuals / len(manuals), 3)

    @staticmethod
    def chunk_similarity_variance(scores: List[float]) -> float:
        """
        Consistency: quanto sono simili tra loro i chunks?
        Bassa varianza = chunks coerenti

        Returns:
            float: 1 - std(scores) [0, 1]
        """
        if len(scores) < 2:
            return 1.0

        return round(1 - np.std(scores), 3)

    @staticmethod
    def average_chunk_score(scores: List[float]) -> float:
        """
        Score medio di similarity

        Returns:
            float: Mean similarity [0, 1]
        """
        if not scores:
            return 0.0

        # Convert distance to similarity se necessario
        similarities = [1 - min(s, 1.0) for s in scores]

        return round(np.mean(similarities), 3)

    @staticmethod
    def calculate_all(retrieved_docs: List, scores: List[float]) -> Dict:
        """
        Calcola tutte le metriche in un colpo

        Returns:
            dict: All retrieval metrics
        """
        return {
            'diversity': RetrievalMetrics.diversity_score(retrieved_docs),
            'consistency': RetrievalMetrics.chunk_similarity_variance(scores),
            'avg_similarity': RetrievalMetrics.average_chunk_score(scores),
            'num_chunks': len(retrieved_docs)
        }

print("✅ RetrievalMetrics loaded")

# ==========================================
# 4. EVALUATION LOGGER (AUTO-UPDATE REPORT)
# ==========================================

@dataclass
class QueryLogEntry:
    """Struttura log entry per JSONL"""
    timestamp: str
    query: str
    response: str
    vehicle: Optional[str]
    confidence: Dict
    llm_judge: Dict
    retrieval_metrics: Dict
    performance: Dict
    clip_used: bool
    injection_detected: bool
    images_found: int
    images_shown: int
    language: str

class EvaluationLogger:
    """
    Logger persistente su Google Drive con AUTO-UPDATE del report
    """

    def __init__(self):
        self.queries_log = PathConfig.QUERIES_LOG
        self.summary_log = PathConfig.SUMMARY_LOG
        self.report_path = PathConfig.EVALUATION_REPORT

    def log_query(self, data: Dict):
        """
        Appendi query al log JSONL e RIGENERA automaticamente il report

        Args:
            data: Dictionary con tutti i campi del log
        """
        # Crea entry strutturata
        entry = QueryLogEntry(
            timestamp=datetime.now().isoformat(),
            query=data.get('query', ''),
            response=data.get('response', ''),
            vehicle=data.get('vehicle'),
            confidence=data.get('confidence', {}),
            llm_judge=data.get('llm_judge', {}),
            retrieval_metrics=data.get('retrieval_metrics', {}),
            performance=data.get('performance', {}),
            clip_used=data.get('clip_used', False),
            injection_detected=data.get('injection_detected', False),
            images_found=data.get('images_found', 0),
            images_shown=data.get('images_shown', 0),
            language=data.get('language', 'unknown')
        )

        # Appendi a JSONL
        with open(self.queries_log, 'a', encoding='utf-8') as f:
            f.write(json.dumps(asdict(entry), ensure_ascii=False) + '\n')

        logger.info(f"Query logged: {data['query'][:50]}...")

        # ✅ AUTO-REGENERATE REPORT
        self.generate_markdown_report()

    def load_all_queries(self) -> List[Dict]:
        """
        Carica tutte le query dal log

        Returns:
            List of query dictionaries
        """
        if not self.queries_log.exists():
            return []

        queries = []
        with open(self.queries_log, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    queries.append(json.loads(line))
                except json.JSONDecodeError:
                    continue

        return queries

    def compute_summary(self) -> Dict:
        """
        Calcola statistiche aggregate da queries.jsonl

        Returns:
            dict: Summary statistics
        """
        queries = self.load_all_queries()

        if not queries:
            return {
                'total_queries': 0,
                'message': 'No data collected yet'
            }

        # Estrai metriche
        confidences = []
        judge_scores = []
        response_times = []
        rag_times = []
        llm_times = []
        img_times = []
        clip_used_count = 0
        injection_count = 0
        retrieval_metrics = {'diversity': [], 'consistency': [], 'avg_similarity': []}

        for q in queries:
            # Confidence
            conf = q.get('confidence', {})
            if 'score' in conf:
                confidences.append(conf['score'])

            # LLM Judge
            judge = q.get('llm_judge', {})
            if 'average' in judge and judge['average'] > 0:
                judge_scores.append(judge['average'])

            # Performance
            perf = q.get('performance', {})
            response_times.append(perf.get('total_time_ms', 0))
            rag_times.append(perf.get('rag_time_ms', 0))
            llm_times.append(perf.get('llm_time_ms', 0))
            img_times.append(perf.get('img_time_ms', 0))

            # CLIP
            if q.get('clip_used', False):
                clip_used_count += 1

            # Security
            if q.get('injection_detected', False):
                injection_count += 1

            # Retrieval
            ret_metrics = q.get('retrieval_metrics', {})
            for key in retrieval_metrics:
                if key in ret_metrics:
                    retrieval_metrics[key].append(ret_metrics[key])

        # Aggregate
        summary = {
            'total_queries': len(queries),
            'date_range': {
                'start': queries[0]['timestamp'],
                'end': queries[-1]['timestamp']
            },
            'confidence': {
                'mean': round(np.mean(confidences), 3) if confidences else 0,
                'std': round(np.std(confidences), 3) if confidences else 0,
                'min': round(min(confidences), 3) if confidences else 0,
                'max': round(max(confidences), 3) if confidences else 0,
                'samples': len(confidences)
            },
            'llm_judge': {
                'mean': round(np.mean(judge_scores), 3) if judge_scores else 0,
                'std': round(np.std(judge_scores), 3) if judge_scores else 0,
                'samples': len(judge_scores)
            },
            'retrieval': {
                'diversity': {
                    'mean': round(np.mean(retrieval_metrics['diversity']), 3) if retrieval_metrics['diversity'] else 0,
                    'std': round(np.std(retrieval_metrics['diversity']), 3) if retrieval_metrics['diversity'] else 0
                },
                'consistency': {
                    'mean': round(np.mean(retrieval_metrics['consistency']), 3) if retrieval_metrics['consistency'] else 0,
                    'std': round(np.std(retrieval_metrics['consistency']), 3) if retrieval_metrics['consistency'] else 0
                },
                'avg_similarity': {
                    'mean': round(np.mean(retrieval_metrics['avg_similarity']), 3) if retrieval_metrics['avg_similarity'] else 0,
                    'std': round(np.std(retrieval_metrics['avg_similarity']), 3) if retrieval_metrics['avg_similarity'] else 0
                }
            },
            'performance': {
                'avg_total_time_ms': round(np.mean(response_times), 1) if response_times else 0,
                'avg_rag_time_ms': round(np.mean(rag_times), 1) if rag_times else 0,
                'avg_llm_time_ms': round(np.mean(llm_times), 1) if llm_times else 0,
                'avg_img_time_ms': round(np.mean(img_times), 1) if img_times else 0,
                'clip_usage_rate': round(clip_used_count / len(queries), 3) if queries else 0
            },
            'security': {
                'injection_attempts': injection_count,
                'block_rate': round(injection_count / len(queries), 3) if queries else 0
            }
        }

        # Salva summary
        with open(self.summary_log, 'w', encoding='utf-8') as f:
            json.dump(summary, f, indent=2, ensure_ascii=False)

        return summary

    def generate_markdown_report(self) -> str:
        """
        Genera report Markdown AUTO-AGGIORNATO con:
        - Stats generali aggregate
        - Last 10 queries dettagliate

        Returns:
            str: Path del report
        """
        queries = self.load_all_queries()

        if not queries:
            report = "# AutoMATE Evaluation Report\n\n**No data collected yet.**\n\nRun at least 1 query to generate the report."
            with open(self.report_path, 'w', encoding='utf-8') as f:
                f.write(report)
            return str(self.report_path)

        summary = self.compute_summary()

        # Get last 10 queries
        last_10 = queries[-10:]

        report = f"""# AutoMATE Evaluation Report
**Automotive Multimodal Augmented Technical Expert**

*Auto-generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*

---

## 📊 Executive Summary

- **Total Queries**: {summary['total_queries']}
- **Period**: {summary['date_range']['start'][:10]} to {summary['date_range']['end'][:10]}
- **System**: Dolphin OCR + Gemini 2.0 Flash + CLIP ViT-B/32

---

## 🎯 Aggregate Performance Metrics

### Confidence Scores (Multi-source)

| Metric | Value |
|--------|-------|
| **Mean Confidence** | {summary['confidence']['mean']} ± {summary['confidence']['std']} |
| **Range** | [{summary['confidence']['min']}, {summary['confidence']['max']}] |
| **Samples** | {summary['confidence']['samples']}/{summary['total_queries']} |

**Interpretation:**
- `≥ 0.75`: **HIGH** confidence (system very confident in answer)
- `0.55-0.75`: **MEDIUM** confidence (reliable answer with caution)
- `< 0.55`: **LOW** confidence (uncertain answer, requires manual verification)

### LLM-as-Judge Evaluation

| Dimension | Score |
|-----------|-------|
| **Average Score** | {summary['llm_judge']['mean']}/5.0 ± {summary['llm_judge']['std']} |
| **Samples** | {summary['llm_judge']['samples']}/{summary['total_queries']} |

### Retrieval Performance

| Metric | Mean ± Std |
|--------|-----------|
| **Diversity** | {summary['retrieval']['diversity']['mean']} ± {summary['retrieval']['diversity']['std']} |
| **Consistency** | {summary['retrieval']['consistency']['mean']} ± {summary['retrieval']['consistency']['std']} |
| **Avg Similarity** | {summary['retrieval']['avg_similarity']['mean']} ± {summary['retrieval']['avg_similarity']['std']} |

### System Performance

| Metric | Value |
|--------|-------|
| **Avg Response Time** | {summary['performance']['avg_total_time_ms']:.0f}ms |
| **Avg RAG Time** | {summary['performance']['avg_rag_time_ms']:.0f}ms |
| **Avg LLM Time** | {summary['performance']['avg_llm_time_ms']:.0f}ms |
| **Avg Image Time** | {summary['performance']['avg_img_time_ms']:.0f}ms |
| **CLIP Usage Rate** | {summary['performance']['clip_usage_rate']*100:.1f}% |

### Security & Safety

| Metric | Value |
|--------|-------|
| **Injection Attempts** | {summary['security']['injection_attempts']} |
| **Block Rate** | {summary['security']['block_rate']*100:.1f}% |

---

## 🔍 Last 10 Queries (Detailed)

"""

        # Add detailed info for last 10 queries
        for idx, q in enumerate(reversed(last_10), 1):
            timestamp = q.get('timestamp', 'N/A')
            query_text = q.get('query', 'N/A')
            vehicle = q.get('vehicle', 'Generic')
            language = q.get('language', 'unknown')

            # Confidence
            conf = q.get('confidence', {})
            conf_score = conf.get('score', 0)
            conf_label = conf.get('label', 'N/A')
            conf_breakdown = conf.get('breakdown', {})

            # LLM Judge
            judge = q.get('llm_judge', {})
            judge_avg = judge.get('average', 0)
            judge_faith = judge.get('faithfulness', 0)
            judge_rel = judge.get('relevance', 0)
            judge_comp = judge.get('completeness', 0)
            judge_reason = judge.get('reasoning', 'N/A')

            # Retrieval metrics
            ret = q.get('retrieval_metrics', {})
            diversity = ret.get('diversity', 0)
            consistency = ret.get('consistency', 0)
            avg_sim = ret.get('avg_similarity', 0)
            num_chunks = ret.get('num_chunks', 0)

            # Performance
            perf = q.get('performance', {})
            total_time = perf.get('total_time_ms', 0)
            rag_time = perf.get('rag_time_ms', 0)
            llm_time = perf.get('llm_time_ms', 0)
            img_time = perf.get('img_time_ms', 0)

            # Images
            img_found = q.get('images_found', 0)
            img_shown = q.get('images_shown', 0)
            clip_used = "✅" if q.get('clip_used', False) else "❌"

            report += f"""### Query #{len(queries) - len(last_10) + idx}: {query_text[:80]}{"..." if len(query_text) > 80 else ""}

**Timestamp:** {timestamp}
**Vehicle:** {vehicle} | **Language:** {language}

#### 🎯 Confidence Score: {conf_label} ({conf_score})

| Component | Score |
|-----------|-------|
| Retrieval | {conf_breakdown.get('retrieval', 0)} |
| Relevance | {conf_breakdown.get('relevance', 0)} |
| Quality | {conf_breakdown.get('quality', 0)} |

#### 📚 Retrieval Metrics

| Metric | Value |
|--------|-------|
| Diversity | {diversity} |
| Consistency | {consistency} |
| Avg Similarity | {avg_sim} |
| Chunks Retrieved | {num_chunks} |

"""

            # Add LLM Judge if available
            if judge_avg > 0:
                report += f"""#### 🤖 LLM Judge Evaluation: {judge_avg}/5.0

| Dimension | Score |
|-----------|-------|
| Faithfulness | {judge_faith}/5 ⭐ |
| Relevance | {judge_rel}/5 ⭐ |
| Completeness | {judge_comp}/5 ⭐ |

**Reasoning:** {judge_reason}

"""

            report += f"""#### ⏱️ Performance

| Metric | Value |
|--------|-------|
| Total Time | {total_time:.0f}ms |
| RAG Time | {rag_time:.0f}ms |
| LLM Time | {llm_time:.0f}ms |
| Image Time | {img_time:.0f}ms |

#### 🖼️ Images

| Metric | Value |
|--------|-------|
| Found | {img_found} |
| Shown | {img_shown} |
| CLIP Used | {clip_used} |

---

"""

        report += f"""
*Report auto-generated by AutoMATE Evaluation Framework v3.0*

*Full query log: `{self.queries_log}`*
"""

        # Save report
        with open(self.report_path, 'w', encoding='utf-8') as f:
            f.write(report)

        logger.info(f"Report auto-updated: {self.report_path}")

        return str(self.report_path)

print("✅ EvaluationLogger loaded (AUTO-UPDATE)")

# ==========================================
# 5. PROMPT PROTECTION
# ==========================================

class PromptProtection:
    """Security contro injection attacks"""

    @staticmethod
    def sanitize_query(query: str) -> Tuple[str, bool]:
        original = query
        patterns = [
            (r'<\|[^>]*\|>', ''),
            (r'system\s*:', '', re.IGNORECASE),
            (r'ignore\s+(previous|all|above|instructions?)', '', re.IGNORECASE),
            (r'forget\s+(previous|all|above|instructions?)', '', re.IGNORECASE),
            (r'you\s+are\s+now', '', re.IGNORECASE),
            (r'act\s+as\s+if', '', re.IGNORECASE),
        ]
        cleaned = query
        for pattern, replacement, *flags in patterns:
            flag = flags[0] if flags else 0
            cleaned = re.sub(pattern, replacement, cleaned, flags=flag)
        cleaned = ' '.join(cleaned.split()).strip()
        return cleaned, (cleaned != original)

print("✅ PromptProtection loaded")

# ==========================================
# SUMMARY
# ==========================================

print("\n" + "=" * 70)
print("🎉 BLOCCO 2 COMPLETATO (v3.0 - AUTO-UPDATE)")
print("=" * 70)

print(f"""
✅ Componenti:
   1. ConfidenceCalculator - Multi-source confidence scoring
   2. LLMJudge - Gemini as arbitrator (AIMessage compatible)
   3. RetrievalMetrics - Intrinsic evaluation (no ground truth)
   4. EvaluationLogger - AUTO-UPDATE report after each query
   5. PromptProtection - Security test suite

📊 Report Features:
   • Stats generali aggregate (dall'inizio)
   • Last 10 queries con TUTTE le metriche
   • Auto-rigenerato dopo OGNI query

📁 Files su Drive:
   {PathConfig.EVALUATION_BASE}/
   ├── logs/queries.jsonl (log completo)
   ├── logs/metrics_summary.json (stats aggregate)
   └── reports/automate_evaluation_report.md (AUTO-UPDATED!)

💡 Il report si aggiorna automaticamente dopo ogni query!
""")

print("=" * 70)

#Advanced Rag Components

In [ ]:
# ==========================================
# BLOCCO 2.5: ADVANCED RAG COMPONENTS (FIXED v3.0)
# ==========================================

import re
import os
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass
import logging

logger = logging.getLogger(__name__)

print("🚀 BLOCCO 2.5: Advanced RAG Components (FIXED v3.0)")
print("=" * 70)

# ==========================================
# VERIFICA BLOCCO 1
# ==========================================

print("\n🔍 Verifica prerequisiti...")

try:
    _test_config = IMAGE_QUALITY_CONFIG
    print(f"✅ IMAGE_QUALITY_CONFIG trovato")
    print(f"   Threshold: {IMAGE_QUALITY_CONFIG['min_width']}x{IMAGE_QUALITY_CONFIG['min_height']}px, "
          f"{IMAGE_QUALITY_CONFIG['min_size_kb']}KB, ratio<{IMAGE_QUALITY_CONFIG['max_aspect_ratio']}")
except NameError:
    print("❌ ERRORE: BLOCCO 1 (Config) non eseguito!")
    raise

# ==========================================
# VEHICLE DETECTOR
# ==========================================

class VehicleDetector:
    """Named Entity Recognition per veicoli"""

    VEHICLE_MAPPING = {
        'peugeot 208': 'PEUGEOT 208',
        '208': 'PEUGEOT 208',
        'peugeot': 'PEUGEOT 208',
        'fiat 500': '500',
        '500': '500',
        'cinquecento': '500',
        'fiat cinquecento': '500',
        'panda': 'PANDA',
        'fiat panda': 'PANDA',
        'grande punto': 'GRANDE-PUNTO',
        'punto': 'GRANDE-PUNTO',
        'fiat punto': 'GRANDE-PUNTO',
        'fiat grande punto': 'GRANDE-PUNTO',
    }

    @classmethod
    def detect(cls, query: str) -> Optional[str]:
        query_lower = query.lower()
        sorted_variants = sorted(cls.VEHICLE_MAPPING.keys(), key=len, reverse=True)
        for variant in sorted_variants:
            if variant in query_lower:
                manual_name = cls.VEHICLE_MAPPING[variant]
                logger.info(f"Vehicle detected: '{variant}' → {manual_name}")
                return manual_name
        return None

    @classmethod
    def get_all_manuals(cls) -> List[str]:
        return list(set(cls.VEHICLE_MAPPING.values()))

print("✅ VehicleDetector loaded")

# ==========================================
# IMAGE QUALITY FILTER
# ==========================================

@dataclass
class ImageQualityThreshold:
    """Threshold per quality filtering - eredita da BLOCCO 1"""
    min_width: int = IMAGE_QUALITY_CONFIG['min_width']
    min_height: int = IMAGE_QUALITY_CONFIG['min_height']
    min_size_kb: float = IMAGE_QUALITY_CONFIG['min_size_kb']
    max_aspect_ratio: float = IMAGE_QUALITY_CONFIG['max_aspect_ratio']

class ImageQualityFilter:
    """Post-processing per escludere immagini di bassa qualità"""

    @staticmethod
    def is_quality_image(img_path: str,
                         threshold: ImageQualityThreshold = None) -> Tuple[bool, str]:
        """
        Verifica se immagine rispetta standard di qualità

        Returns:
            (is_quality, reason)
        """
        if threshold is None:
            threshold = ImageQualityThreshold()

        try:
            if not os.path.exists(img_path):
                return False, "FILE_NOT_FOUND"

            size_kb = os.path.getsize(img_path) / 1024
            if size_kb < threshold.min_size_kb:
                return False, f"TOO_SMALL_FILE ({size_kb:.1f}KB < {threshold.min_size_kb}KB)"

            from PIL import Image
            with Image.open(img_path) as img:
                width, height = img.size

                if width < threshold.min_width or height < threshold.min_height:
                    return False, f"LOW_RESOLUTION ({width}x{height})"

                aspect_ratio = max(width, height) / min(width, height)
                if aspect_ratio > threshold.max_aspect_ratio:
                    return False, f"BAD_ASPECT_RATIO ({aspect_ratio:.1f})"

            return True, "OK"

        except Exception as e:
            return False, f"ERROR: {str(e)[:50]}"

    @staticmethod
    def filter_results(img_results: List,
                       threshold: ImageQualityThreshold = None) -> List:
        """Filtra lista di risultati immagini"""
        if threshold is None:
            threshold = ImageQualityThreshold()

        filtered = []
        rejected_count = 0

        for result in img_results:
            doc = result[0]
            img_path = doc.metadata.get('image_path')
            is_quality, reason = ImageQualityFilter.is_quality_image(img_path, threshold)

            if is_quality:
                filtered.append(result)
            else:
                rejected_count += 1
                logger.debug(f"Rejected: {reason}")

        if rejected_count > 0:
            logger.info(f"Quality filter: {len(filtered)}/{len(img_results)} passed")

        return filtered

print("✅ ImageQualityFilter loaded")
print(f"   Threshold: {IMAGE_QUALITY_CONFIG['min_width']}x{IMAGE_QUALITY_CONFIG['min_height']}px, {IMAGE_QUALITY_CONFIG['min_size_kb']}KB")

# ==========================================
# PROMPT PROTECTION
# ==========================================

class PromptProtection:
    """Security contro injection attacks"""

    @staticmethod
    def sanitize_query(query: str) -> Tuple[str, bool]:
        original = query
        patterns = [
            (r'<\|[^>]*\|>', ''),
            (r'system\s*:', '', re.IGNORECASE),
            (r'ignore\s+(previous|all|above|instructions?)', '', re.IGNORECASE),
            (r'forget\s+(previous|all|above|instructions?)', '', re.IGNORECASE),
            (r'you\s+are\s+now', '', re.IGNORECASE),
            (r'act\s+as\s+if', '', re.IGNORECASE),
        ]
        cleaned = query
        for pattern, replacement, *flags in patterns:
            flag = flags[0] if flags else 0
            cleaned = re.sub(pattern, replacement, cleaned, flags=flag)
        cleaned = ' '.join(cleaned.split()).strip()
        return cleaned, (cleaned != original)

print("✅ PromptProtection loaded")

# ==========================================
# EVALUATION LOGGER
# ==========================================

class EvaluationLogger:
    """Logger per salvare risultati su Drive"""

    def __init__(self):
        from pathlib import Path
        self.log_path = Path("/content/drive/MyDrive/OCR/evaluation/logs/queries.jsonl")
        self.report_path = Path("/content/drive/MyDrive/OCR/evaluation/reports/evaluation_report.md")
        self.log_path.parent.mkdir(parents=True, exist_ok=True)
        self.report_path.parent.mkdir(parents=True, exist_ok=True)

    def log_query(self, log_entry: Dict):
        import json
        with open(self.log_path, 'a', encoding='utf-8') as f:
            f.write(json.dumps(log_entry, ensure_ascii=False) + '\n')

    def generate_markdown_report(self) -> str:
        import json
        from datetime import datetime

        queries = []
        if self.log_path.exists():
            with open(self.log_path, 'r', encoding='utf-8') as f:
                for line in f:
                    try:
                        queries.append(json.loads(line))
                    except:
                        pass

        with open(self.report_path, 'w', encoding='utf-8') as f:
            f.write(f"# Evaluation Report\n\n")
            f.write(f"**Generated**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            f.write(f"**Total Queries**: {len(queries)}\n")

        return str(self.report_path)

print("✅ EvaluationLogger loaded")

eval_logger = EvaluationLogger()

# ==========================================
# VEHICLE AWARE RETRIEVER
# ==========================================

class VehicleAwareRetriever:
    """Wrapper per retrieval con filtering per manuale"""

    def __init__(self, text_db, img_db):
        self.text_db = text_db
        self.img_db = img_db

    def retrieve_text(self, query: str, k: int = 6,
                      manual_filter: Optional[str] = None) -> List:
        if manual_filter:
            results = self.text_db.similarity_search_with_score(
                query, k=k*3, filter={"manual": manual_filter}
            )
            return results[:k]
        else:
            return self.text_db.similarity_search_with_score(query, k=k)

    def retrieve_images(self, query: str, stage1_k: int = 20,
                        manual_filter: Optional[str] = None) -> List:
        if manual_filter:
            results = self.img_db.similarity_search_with_score(
                query, k=stage1_k*3, filter={"manual": manual_filter}
            )
            return results[:stage1_k]
        else:
            return self.img_db.similarity_search_with_score(query, k=stage1_k)

print("✅ VehicleAwareRetriever loaded")

# ==========================================
# SUMMARY
# ==========================================

print("\n" + "=" * 70)
print("🎉 BLOCCO 2.5 COMPLETATO (FIXED v3.0)")
print("=" * 70)

print(f"""
✅ Componenti:
   1. VehicleDetector - NER per veicoli
   2. ImageQualityFilter - Filtra immagini spurie
   3. PromptProtection - Security
   4. EvaluationLogger - Logging su Drive
   5. VehicleAwareRetriever - Retrieval filtrato

📊 Threshold attuali (da BLOCCO 1):
   • Resolution: {IMAGE_QUALITY_CONFIG['min_width']}x{IMAGE_QUALITY_CONFIG['min_height']}px
   • Size: {IMAGE_QUALITY_CONFIG['min_size_kb']}KB
   • Aspect ratio: <{IMAGE_QUALITY_CONFIG['max_aspect_ratio']}

💡 Per modificare threshold: cambia BLOCCO 1
""")

#Test RAG

In [ ]:
# ==========================================
# BLOCCO 3: TEST RAG SYSTEM (FIXED + IMAGE QUERY AUGMENTATION)
# ==========================================

import os
import time
import gc
import torch
import json
import numpy as np
from pathlib import Path

print("🚀 BLOCCO 3: Test RAG System (FIXED + IMAGE QUERY AUGMENTATION)")
print("=" * 70)

# ==========================================
# VERIFICA PREREQUISITI
# ==========================================

print("\n🔍 Verifica prerequisiti...")

try:
    _test = IMAGE_QUALITY_CONFIG
    print("✅ BLOCCO 1 (Config)")
except NameError:
    print("❌ BLOCCO 1 non eseguito!")
    raise

try:
    _test = ConfidenceCalculator
    print("✅ BLOCCO 2 (Evaluation)")
except NameError:
    print("❌ BLOCCO 2 non eseguito!")
    raise

try:
    _test = VehicleDetector
    print("✅ BLOCCO 2.5 (Advanced RAG)")
except NameError:
    print("❌ BLOCCO 2.5 non eseguito!")
    raise

# ==========================================
# CONFIGURAZIONE TEST
# ==========================================

QUERY = "Come funziona il pulsante TRIP della fiat 500?"

ENABLE_LLM_JUDGE = True
ENABLE_INJECTION_TEST = True

# ✅ NEW: Query augmentation per immagini
ENABLE_IMAGE_QUERY_AUGMENTATION = True

quality_threshold = ImageQualityThreshold()

print(f"""
{"="*70}
⚙️ CONFIGURAZIONE TEST (v3.2 - IMAGE QUERY AUGMENTATION):
{"="*70}

📝 Query: "{QUERY}"

🖼️ Quality Filter (da BLOCCO 1):
   • Resolution: {quality_threshold.min_width}x{quality_threshold.min_height}px
   • Size: {quality_threshold.min_size_kb}KB

🔬 Evaluation:
   • LLM Judge: {'✅ ENABLED' if ENABLE_LLM_JUDGE else '❌ DISABLED'}

🆕 Image Query Augmentation: {'✅ ENABLED' if ENABLE_IMAGE_QUERY_AUGMENTATION else '❌ DISABLED'}
   → Espande la query per trovare immagini più rilevanti
{"="*70}
""")

# ==========================================
# CARICAMENTO RISORSE
# ==========================================

print("\n⚙️ CARICAMENTO RISORSE...")

from langchain_huggingface import HuggingFaceEmbeddings
try:
    from langchain_chroma import Chroma
except ImportError:
    from langchain_community.vectorstores import Chroma

print("  1️⃣ Text Embeddings...")
text_embed = HuggingFaceEmbeddings(
    model_name=TEXT_EMBED_MODEL,
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True}
)
print(f"     ✅ {TEXT_EMBED_MODEL}")

print("  2️⃣ Vector Stores...")
text_db = Chroma(
    collection_name=TEXT_COLLECTION_NAME,
    persist_directory=CHROMA_TEXT_DIR,
    embedding_function=text_embed
)
print(f"     ✅ Text: {text_db._collection.count()} documenti")

img_db = Chroma(
    collection_name=IMAGE_COLLECTION_NAME,
    persist_directory=CHROMA_IMAGE_DIR,
    embedding_function=text_embed
)
print(f"     ✅ Images: {img_db._collection.count()} immagini")

print("  3️⃣ CLIP...")
with open(CLIP_JSON_PATH, 'r') as f:
    clip_embeddings = json.load(f)
print(f"     ✅ CLIP embeddings: {len(clip_embeddings)}")

from transformers import CLIPModel, CLIPProcessor
clip_model = CLIPModel.from_pretrained(CLIP_MODEL_NAME).to('cuda')
clip_processor = CLIPProcessor.from_pretrained(CLIP_MODEL_NAME)
print(f"     ✅ CLIP model: {CLIP_MODEL_NAME}")

print("  4️⃣ Gemini LLM...")
from langchain_google_genai import ChatGoogleGenerativeAI, HarmCategory, HarmBlockThreshold

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.1,
    safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    }
)
print("     ✅ Gemini 2.0 Flash Lite")

print("\n✅ Risorse caricate!")

# ==========================================
# QUERY AUGMENTATION FUNCTION
# ==========================================

def augment_query_for_images(query: str, llm_instance) -> str:
    """
    Usa Gemini per espandere la query con termini visivi/tecnici
    che aiutano a trovare immagini rilevanti
    """
    augment_prompt = f"""Data questa domanda su un manuale auto:
"{query}"

Genera una lista di 5-8 parole chiave che potrebbero apparire nella didascalia o nel contesto di un'immagine rilevante.
Includi: componenti visibili, posizioni (cruscotto, volante, display), azioni (premere, visualizzare).

Rispondi SOLO con le parole chiave separate da spazi, senza punteggiatura o spiegazioni.
Esempio: pulsante cruscotto display visualizzazione reset azzeramento"""

    try:
        result = llm_instance.invoke(augment_prompt)
        keywords = result.content if hasattr(result, 'content') else str(result)
        keywords = keywords.strip().replace('\n', ' ').replace(',', ' ')

        # Combina query originale + keywords
        augmented = f"{query} {keywords}"
        return augmented
    except Exception as e:
        print(f"     ⚠️ Augmentation fallita: {e}")
        return query

# ==========================================
# SECURITY CHECK
# ==========================================

if ENABLE_INJECTION_TEST:
    sanitized_query, was_modified = PromptProtection.sanitize_query(QUERY)
    if was_modified:
        print(f"\n⚠️ Query sanitizzata: {sanitized_query}")
    else:
        print("\n✅ Query pulita")
else:
    sanitized_query = QUERY

# ==========================================
# VEHICLE DETECTION
# ==========================================

detected_vehicle = VehicleDetector.detect(sanitized_query)

print(f"\n🚗 VEHICLE DETECTION")
print("-"*70)
if detected_vehicle:
    print(f"✅ Veicolo rilevato: {detected_vehicle}")
else:
    print("ℹ️ Nessun veicolo (query generica)")

# ==========================================
# TEXT RAG
# ==========================================

print(f"\n🤖 ELABORAZIONE RAG TESTUALE")
print("-"*70)

start_time = time.time()

# 1. Retrieval
print("📚 Retrieval chunks...")
if detected_vehicle:
    docs_with_scores = text_db.similarity_search_with_score(
        sanitized_query, k=18, filter={"manual": detected_vehicle}
    )[:6]
else:
    docs_with_scores = text_db.similarity_search_with_score(sanitized_query, k=6)

retrieved_docs = [doc for doc, _ in docs_with_scores]
print(f"   ✅ {len(retrieved_docs)} chunks retrieved")

# 2. Metrics
scores_only = [score for _, score in docs_with_scores]
retrieval_metrics = RetrievalMetrics.calculate_all(retrieved_docs, scores_only)
print(f"   • Diversity: {retrieval_metrics['diversity']:.3f}")
print(f"   • Consistency: {retrieval_metrics['consistency']:.3f}")

# 3. Confidence
retrieval_conf = ConfidenceCalculator.retrieval_confidence(docs_with_scores)
relevance_conf = ConfidenceCalculator.context_relevance(sanitized_query, retrieved_docs, text_embed)
print(f"   • Retrieval conf: {retrieval_conf}")
print(f"   • Relevance conf: {relevance_conf}")

# 4. Context
def format_docs(docs):
    parts = []
    for i, doc in enumerate(docs, 1):
        parts.append(f"[Sezione {i}]\n{doc.page_content.strip()}")
    return "\n\n---\n\n".join(parts)

context_text = format_docs(retrieved_docs)

# 5. RAG Chain (FIXED prompt)
print("🔗 Creazione RAG chain...")

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

if detected_vehicle:
    template = """Sei un esperto tecnico specializzato nel manuale {vehicle}.

ISTRUZIONI:
1. Rispondi SOLO usando le informazioni nel CONTESTO
2. Se l'informazione non è nel contesto, dillo chiaramente
3. Includi TUTTI i dettagli tecnici: procedure, valori, avvertenze
4. Rispondi in italiano

CONTESTO DAL MANUALE {vehicle}:
{context}

DOMANDA:
{question}

RISPOSTA TECNICA DETTAGLIATA:"""

    prompt = PromptTemplate(template=template, input_variables=["context", "question", "vehicle"])

    chain = (
        {
            "context": RunnableLambda(lambda _: context_text),
            "question": RunnablePassthrough(),
            "vehicle": RunnableLambda(lambda _: detected_vehicle)
        }
        | prompt | llm | StrOutputParser()
    )
else:
    template = """Sei un esperto tecnico di manuali automotive.

ISTRUZIONI:
1. Rispondi usando le informazioni nel CONTESTO
2. Se non trovi l'informazione, dillo
3. Includi dettagli tecnici
4. Rispondi in italiano

CONTESTO:
{context}

DOMANDA:
{question}

RISPOSTA:"""

    prompt = PromptTemplate(template=template, input_variables=["context", "question"])

    chain = (
        {
            "context": RunnableLambda(lambda _: context_text),
            "question": RunnablePassthrough()
        }
        | prompt | llm | StrOutputParser()
    )

# 6. Generate
print("💬 Generazione risposta...")
llm_start = time.time()
response = chain.invoke(sanitized_query)
llm_time = (time.time() - llm_start) * 1000
print(f"   ✅ Risposta generata ({llm_time:.1f}ms)")

# 7. Quality confidence
quality_conf = ConfidenceCalculator.answer_quality(response)
print(f"   • Answer quality: {quality_conf}")

rag_time = (time.time() - start_time) * 1000

confidence = ConfidenceCalculator.aggregate(retrieval_conf, relevance_conf, quality_conf)

# ==========================================
# DISPLAY RESPONSE
# ==========================================

print("\n" + "="*70)
print("📖 RISPOSTA")
if detected_vehicle:
    print(f"🚗 Manuale: {detected_vehicle}")
print("="*70)
print("\n" + response + "\n")

emoji = '🟢' if confidence['label'] == 'HIGH' else ('🟡' if confidence['label'] == 'MEDIUM' else '🔴')
print("="*70)
print(f"{emoji} CONFIDENCE: {confidence['label']} ({confidence['score']})")
print("="*70)
print(f"   🔍 Retrieval: {confidence['breakdown']['retrieval']:.3f}")
print(f"   🎯 Relevance: {confidence['breakdown']['relevance']:.3f}")
print(f"   ✍️ Quality: {confidence['breakdown']['quality']:.3f}")

# ==========================================
# LLM JUDGE
# ==========================================

judge_data = None
if ENABLE_LLM_JUDGE:
    print("\n" + "="*70)
    print("🤖 LLM JUDGE EVALUATION")
    print("-"*70)

    judge = LLMJudge(llm)
    judge_data = judge.evaluate_response(sanitized_query, context_text, response)

    print(f"   Faithfulness:  {judge_data['faithfulness']}/5  {'⭐' * judge_data['faithfulness']}")
    print(f"   Relevance:     {judge_data['relevance']}/5  {'⭐' * judge_data['relevance']}")
    print(f"   Completeness:  {judge_data['completeness']}/5  {'⭐' * judge_data['completeness']}")
    print(f"\n   🏆 Average: {judge_data['average']:.2f}/5")

# ==========================================
# IMAGE RETRIEVAL (WITH QUERY AUGMENTATION)
# ==========================================

print("\n" + "="*70)
print("🖼️ IMAGE RETRIEVAL (con Query Augmentation)")
print("-"*70)

img_start = time.time()

# ✅ NEW: Query augmentation per immagini
if ENABLE_IMAGE_QUERY_AUGMENTATION:
    print("🔧 Augmenting query per image retrieval...")
    image_query = augment_query_for_images(sanitized_query, llm)
    print(f"   Query originale: '{sanitized_query}'")
    print(f"   Query augmented: '{image_query[:100]}...'")
else:
    image_query = sanitized_query

print(f"\n   🚗 Vehicle filter: {detected_vehicle}")
print(f"   📊 Stage 1: Top {STAGE1_TOP_K}")
print(f"   🎯 Stage 2: Top {STAGE2_TOP_K}")

# Stage 1: Text similarity con query augmented
if detected_vehicle:
    img_candidates = img_db.similarity_search_with_score(
        image_query, k=STAGE1_TOP_K*3, filter={"manual": detected_vehicle}
    )[:STAGE1_TOP_K]
else:
    img_candidates = img_db.similarity_search_with_score(image_query, k=STAGE1_TOP_K)

clip_available = sum(1 for doc, _ in img_candidates if doc.metadata.get('has_clip', False))
print(f"\n   ✅ Stage 1: {len(img_candidates)} candidati ({clip_available} con CLIP)")

# Mostra se le immagini con "TRIP" nel content sono nei candidati
trip_in_candidates = sum(1 for doc, _ in img_candidates if 'trip' in doc.page_content.lower())
print(f"   📊 Candidati con 'TRIP' nel content: {trip_in_candidates}/{len(img_candidates)}")

# Stage 2: CLIP re-ranking
inputs = clip_processor(text=[image_query], return_tensors="pt", padding=True)
inputs = {k: v.to('cuda') for k, v in inputs.items()}

with torch.no_grad():
    query_features = clip_model.get_text_features(**inputs)
    query_features = query_features / query_features.norm(dim=-1, keepdim=True)

query_emb = query_features[0].cpu().numpy()

reranked = []
for doc, text_score in img_candidates:
    img_id = doc.metadata.get('image_id')
    has_clip = doc.metadata.get('has_clip', False)

    if has_clip and img_id in clip_embeddings:
        clip_emb = np.array(clip_embeddings[img_id])
        clip_score = np.dot(query_emb, clip_emb) / (np.linalg.norm(query_emb) * np.linalg.norm(clip_emb))
        text_sim = 1 - text_score
        combined = (CLIP_WEIGHT * clip_score) + ((1 - CLIP_WEIGHT) * text_sim)
        reranked.append((doc, combined, True))
    else:
        reranked.append((doc, 1 - text_score, False))

reranked.sort(key=lambda x: x[1], reverse=True)
top_images = reranked[:STAGE2_TOP_K]

print(f"   ✅ Stage 2: Top {len(top_images)}")

# Quality filtering
filtered_images = ImageQualityFilter.filter_results(top_images, quality_threshold)
rejected = len(top_images) - len(filtered_images)
print(f"   ✅ Quality Filter: {len(filtered_images)}/{len(top_images)} ({rejected} rejected)")

img_time = (time.time() - img_start) * 1000

# Display images
print(f"\n   🏆 Top {len(filtered_images)} immagini:")

if filtered_images:
    from PIL import Image as PILImage
    import matplotlib.pyplot as plt

    num_imgs = min(len(filtered_images), 5)
    fig, axes = plt.subplots(1, num_imgs, figsize=(4*num_imgs, 4))

    if num_imgs == 1:
        axes = [axes]

    for idx, (doc, score, used_clip) in enumerate(filtered_images[:num_imgs]):
        img_path = doc.metadata.get('image_path')

        if img_path and os.path.exists(img_path):
            try:
                img = PILImage.open(img_path)
                axes[idx].imshow(img)
                axes[idx].axis('off')

                # Check se ha TRIP nel content
                has_trip = '✓TRIP' if 'trip' in doc.page_content.lower() else ''
                method = "CLIP" if used_clip else "Text"
                w, h = img.size
                axes[idx].set_title(f"#{idx+1} [{method}] {has_trip}\n{w}x{h}px | {score:.3f}", fontsize=9)
            except:
                axes[idx].text(0.5, 0.5, 'Error', ha='center', va='center')
                axes[idx].axis('off')
        else:
            axes[idx].text(0.5, 0.5, 'Not found', ha='center', va='center')
            axes[idx].axis('off')

    plt.tight_layout()
    plt.show()

    print("\n   📝 Dettagli:")
    for idx, (doc, score, used_clip) in enumerate(filtered_images[:num_imgs], 1):
        caption = doc.metadata.get('caption', 'N/A')
        has_trip = "✅ TRIP" if 'trip' in doc.page_content.lower() else "❌"
        method = "CLIP" if used_clip else "Text"

        # Mostra snippet del context
        context_snippet = doc.page_content[:100].replace('\n', ' ')

        print(f"      {idx}. [{method}] {score:.3f} | {has_trip}")
        print(f"         Context: '{context_snippet}...'")
else:
    print("\n   ⚠️ Nessuna immagine trovata")

# Cleanup
del query_features, query_emb, inputs
gc.collect()
torch.cuda.empty_cache()

# ==========================================
# SUMMARY
# ==========================================

total_time = (time.time() - start_time) * 1000

print("\n" + "="*70)
print("📊 SUMMARY")
print("="*70)

print(f"""
⏱️ Performance:
   • Total: {total_time:.1f}ms
   • RAG: {rag_time:.1f}ms
   • LLM: {llm_time:.1f}ms
   • Images: {img_time:.1f}ms

🚗 Vehicle: {detected_vehicle}

🎯 Confidence: {confidence['score']} ({confidence['label']})
""")

if judge_data:
    print(f"🤖 LLM Judge: {judge_data['average']:.2f}/5")

print(f"""
🖼️ Images:
   • Query Augmentation: {'ON' if ENABLE_IMAGE_QUERY_AUGMENTATION else 'OFF'}
   • Candidati con TRIP: {trip_in_candidates}/{len(img_candidates)}
   • After filter: {len(filtered_images)}
""")

print("="*70)
print("✅ TEST COMPLETATO!")
print("="*70)

#Web App streamlit

In [ ]:
# ==========================================
# BLOCCO 4: STREAMLIT APP - AutoMATE (v3.5 - ITALIAN MANUALS OPTIMIZED)
# ==========================================

import os
import time
from pathlib import Path

print("🚀 BLOCCO 4: Streamlit App - AutoMATE (v3.5 - ITALIAN MANUALS OPTIMIZED)")
print("=" * 70)

# ==========================================
# VERIFICA PREREQUISITI
# ==========================================

print("\n🔍 Checking prerequisites...")

try:
    _test_config = IMAGE_QUALITY_CONFIG
    print(f"✅ BLOCCO 1 (Config)")
except NameError:
    print("❌ ERROR: BLOCCO 1 not executed!")
    raise

# ==========================================
# LEGGI VALORI DA CONFIG
# ==========================================

DEFAULT_MIN_WIDTH = int(IMAGE_QUALITY_CONFIG['min_width'])
DEFAULT_MIN_HEIGHT = int(IMAGE_QUALITY_CONFIG['min_height'])
DEFAULT_MIN_SIZE_KB = float(IMAGE_QUALITY_CONFIG['min_size_kb'])
DEFAULT_MAX_ASPECT_RATIO = float(IMAGE_QUALITY_CONFIG['max_aspect_ratio'])

STAGE1_K = int(STAGE1_TOP_K)
STAGE2_K = min(int(STAGE2_TOP_K), 10)
CLIP_W = float(CLIP_WEIGHT)

print(f"✅ Thresholds from BLOCCO 1:")
print(f"   • Resolution: {DEFAULT_MIN_WIDTH}x{DEFAULT_MIN_HEIGHT}px")
print(f"   • Size: {DEFAULT_MIN_SIZE_KB}KB")
print(f"   • Stage 2 default: {STAGE2_K} (range 1-10)")

# ==========================================
# INSTALLAZIONE DIPENDENZE
# ==========================================

print("\n📦 Installing dependencies...")
!pip install -q streamlit pyngrok
print("✅ Dependencies installed")

# ==========================================
# GENERA APP.PY (FULLY EMBEDDED + CROSS-LANGUAGE)
# ==========================================

print("\n📝 Generating app.py (fully embedded + cross-language optimized)...")

app_code = f'''
# ==========================================
# AutoMATE v3.5 - Italian Manuals Optimized
# ==========================================

import streamlit as st
import os
import torch
import gc
import logging
import time
import json
import re
import numpy as np
from datetime import datetime
from pathlib import Path
from dataclasses import dataclass, asdict
from typing import List, Dict, Optional, Tuple

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ==========================================
# EMBEDDED CONFIGURATION
# ==========================================

GOOGLE_API_KEY = "{GOOGLE_API_KEY}"

CHROMA_TEXT_DIR = "{CHROMA_TEXT_DIR}"
CHROMA_IMAGE_DIR = "{CHROMA_IMAGE_DIR}"
CLIP_JSON_PATH = "{CLIP_JSON_PATH}"

TEXT_EMBED_MODEL = "{TEXT_EMBED_MODEL}"
CLIP_MODEL_NAME = "{CLIP_MODEL_NAME}"
TEXT_COLLECTION_NAME = "{TEXT_COLLECTION_NAME}"
IMAGE_COLLECTION_NAME = "{IMAGE_COLLECTION_NAME}"

STAGE1_TOP_K = {STAGE1_K}
STAGE2_TOP_K = {STAGE2_K}
DEFAULT_CLIP_WEIGHT = {CLIP_W}

DEFAULT_MIN_WIDTH = {DEFAULT_MIN_WIDTH}
DEFAULT_MIN_HEIGHT = {DEFAULT_MIN_HEIGHT}
DEFAULT_MIN_SIZE_KB = {DEFAULT_MIN_SIZE_KB}
DEFAULT_MAX_ASPECT_RATIO = {DEFAULT_MAX_ASPECT_RATIO}

EVAL_BASE = Path("/content/drive/MyDrive/OCR/evaluation")
QUERIES_LOG = EVAL_BASE / "logs" / "queries.jsonl"
REPORT_PATH = EVAL_BASE / "reports" / "automate_evaluation_report.md"

# ==========================================
# IMPORTS
# ==========================================

try:
    from langchain_chroma import Chroma
except ImportError:
    from langchain_community.vectorstores import Chroma

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI, HarmCategory, HarmBlockThreshold
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from transformers import CLIPModel, CLIPProcessor
from PIL import Image as PILImage

# ==========================================
# PAGE CONFIG
# ==========================================

st.set_page_config(
    page_title="AutoMATE - Automotive Technical Assistant",
    page_icon="🚗",
    layout="wide",
    initial_sidebar_state="expanded"
)

# ==========================================
# SESSION STATE
# ==========================================

for key in ['resources_loaded', 'text_db', 'img_db', 'clip_embeddings',
            'clip_model', 'clip_processor', 'llm', 'text_embed', 'query_history']:
    if key not in st.session_state:
        st.session_state[key] = None if key != 'resources_loaded' else False
        if key == 'query_history':
            st.session_state[key] = []

# ==========================================
# CSS
# ==========================================

st.markdown("""
<style>
    .main-header {{
        font-size: 2.5rem;
        font-weight: bold;
        background: linear-gradient(90deg, #1f77b4, #2ca02c);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        text-align: center;
        padding: 1rem 0;
    }}
    .subtitle {{
        font-size: 0.9rem;
        color: #666;
        text-align: center;
        font-style: italic;
        margin-top: -15px;
        margin-bottom: 20px;
    }}
    .confidence-high {{
        background: linear-gradient(90deg, #4CAF50, #45a049);
        color: white;
        padding: 12px;
        border-radius: 8px;
        font-weight: bold;
    }}
    .confidence-medium {{
        background: linear-gradient(90deg, #FF9800, #f57c00);
        color: white;
        padding: 12px;
        border-radius: 8px;
        font-weight: bold;
    }}
    .confidence-low {{
        background: linear-gradient(90deg, #F44336, #d32f2f);
        color: white;
        padding: 12px;
        border-radius: 8px;
        font-weight: bold;
    }}
    .vehicle-badge {{
        background: #e3f2fd;
        color: #1976d2;
        padding: 5px 15px;
        border-radius: 20px;
        font-weight: bold;
        display: inline-block;
    }}
    .filter-disabled {{
        background: #fff3e0;
        border: 2px dashed #ff9800;
        padding: 10px;
        border-radius: 8px;
        margin: 10px 0;
    }}
    .augment-enabled {{
        background: #e8f5e9;
        border: 2px solid #4CAF50;
        padding: 10px;
        border-radius: 8px;
        margin: 10px 0;
        font-size: 0.85em;
    }}
    .augment-disabled {{
        background: #fafafa;
        border: 1px dashed #ccc;
        padding: 10px;
        border-radius: 8px;
        margin: 10px 0;
    }}
    .lang-badge {{
        background: #f3e5f5;
        color: #7b1fa2;
        padding: 3px 10px;
        border-radius: 12px;
        font-size: 0.85em;
        font-weight: bold;
        display: inline-block;
    }}
</style>
""", unsafe_allow_html=True)

# ==========================================
# EMBEDDED CLASSES (from BLOCCO 2)
# ==========================================

class ConfidenceCalculator:
    @staticmethod
    def retrieval_confidence(docs_with_scores: List[Tuple]) -> float:
        if not docs_with_scores:
            return 0.0
        scores = [1 - min(score, 1.0) for _, score in docs_with_scores]
        avg = np.mean(scores)
        consistency = 1 - np.std(scores) if len(scores) > 1 else 1.0
        manuals = set(doc.metadata.get('manual', '') for doc, _ in docs_with_scores)
        penalty = 0.95 if len(manuals) > 2 else 1.0
        return round((avg * 0.6 + consistency * 0.4) * penalty, 3)

    @staticmethod
    def context_relevance(query: str, chunks: List, embed_model) -> float:
        if not chunks:
            return 0.0
        try:
            query_emb = np.array(embed_model.embed_query(query))
            relevances = []
            for chunk in chunks:
                chunk_emb = np.array(embed_model.embed_query(chunk.page_content[:500]))
                sim = np.dot(query_emb, chunk_emb) / (np.linalg.norm(query_emb) * np.linalg.norm(chunk_emb))
                relevances.append(sim)
            return round(np.mean(relevances), 3)
        except:
            return 0.5

    @staticmethod
    def answer_quality(response: str) -> float:
        if not response or len(response.strip()) < 10:
            return 0.0
        score = 0.0
        words = response.split()
        length = len(words)
        if 50 < length < 300:
            score += 0.35
        elif 30 < length <= 50:
            score += 0.20
        if re.search(r'\\d+\\s*(km|m|cm|mm|kg|g|bar|V|A|kW|rpm|l)', response):
            score += 0.15
        keywords = ['premere', 'ruotare', 'spegnere', 'accendere', 'verificare', 'controllare',
                   'press', 'turn', 'switch', 'check', 'verify']
        if any(kw in response.lower() for kw in keywords):
            score += 0.10
        sentences = [s for s in response.split('.') if len(s.strip()) > 10]
        if len(sentences) >= 3:
            score += 0.15
        fallback = ['non riesco', 'non posso', 'non ho informazioni', 'cannot', 'unable']
        if not any(phrase in response.lower() for phrase in fallback):
            score += 0.10
        return round(min(score, 1.0), 3)

    @staticmethod
    def aggregate(retrieval: float, relevance: float, quality: float) -> Dict:
        final = retrieval * 0.40 + relevance * 0.35 + quality * 0.25
        if final >= 0.75:
            label, color = "HIGH", "#4CAF50"
        elif final >= 0.55:
            label, color = "MEDIUM", "#FF9800"
        else:
            label, color = "LOW", "#F44336"
        return {{
            'score': round(final, 3),
            'label': label,
            'color': color,
            'breakdown': {{
                'retrieval': round(retrieval, 3),
                'relevance': round(relevance, 3),
                'quality': round(quality, 3)
            }}
        }}


class RetrievalMetrics:
    @staticmethod
    def diversity_score(retrieved_docs: List) -> float:
        if not retrieved_docs:
            return 0.0
        manuals = [doc.metadata.get('manual', 'unknown') for doc in retrieved_docs]
        unique_manuals = len(set(manuals))
        return round(unique_manuals / len(manuals), 3)

    @staticmethod
    def chunk_similarity_variance(scores: List[float]) -> float:
        if len(scores) < 2:
            return 1.0
        return round(1 - np.std(scores), 3)

    @staticmethod
    def average_chunk_score(scores: List[float]) -> float:
        if not scores:
            return 0.0
        similarities = [1 - min(s, 1.0) for s in scores]
        return round(np.mean(similarities), 3)


class VehicleDetector:
    VEHICLE_MAPPING = {{
        'peugeot 208': 'PEUGEOT 208',
        '208': 'PEUGEOT 208',
        'peugeot': 'PEUGEOT 208',
        'fiat 500': '500',
        '500': '500',
        'cinquecento': '500',
        'panda': 'PANDA',
        'fiat panda': 'PANDA',
        'grande punto': 'GRANDE-PUNTO',
        'punto': 'GRANDE-PUNTO',
        'fiat punto': 'GRANDE-PUNTO',
    }}

    @classmethod
    def detect(cls, query: str) -> Optional[str]:
        query_lower = query.lower()
        sorted_variants = sorted(cls.VEHICLE_MAPPING.keys(), key=len, reverse=True)
        for variant in sorted_variants:
            if variant in query_lower:
                return cls.VEHICLE_MAPPING[variant]
        return None


@dataclass
class ImageQualityThreshold:
    min_width: int = DEFAULT_MIN_WIDTH
    min_height: int = DEFAULT_MIN_HEIGHT
    min_size_kb: float = DEFAULT_MIN_SIZE_KB
    max_aspect_ratio: float = DEFAULT_MAX_ASPECT_RATIO


class ImageQualityFilter:
    @staticmethod
    def is_quality_image(img_path: str, threshold: ImageQualityThreshold) -> Tuple[bool, str]:
        try:
            if not os.path.exists(img_path):
                return False, "FILE_NOT_FOUND"
            size_kb = os.path.getsize(img_path) / 1024
            if size_kb < threshold.min_size_kb:
                return False, f"TOO_SMALL ({{size_kb:.1f}}KB)"
            with PILImage.open(img_path) as img:
                width, height = img.size
                if width < threshold.min_width or height < threshold.min_height:
                    return False, f"LOW_RES ({{width}}x{{height}})"
                aspect = max(width, height) / min(width, height)
                if aspect > threshold.max_aspect_ratio:
                    return False, f"BAD_RATIO ({{aspect:.1f}})"
            return True, "OK"
        except:
            return False, "ERROR"

    @staticmethod
    def filter_results(img_results: List, threshold: ImageQualityThreshold, enabled: bool = True) -> List:
        if not enabled:
            return img_results
        filtered = []
        for result in img_results:
            doc = result[0]
            img_path = doc.metadata.get('image_path')
            is_quality, _ = ImageQualityFilter.is_quality_image(img_path, threshold)
            if is_quality:
                filtered.append(result)
        return filtered


class PromptProtection:
    @staticmethod
    def sanitize_query(query: str) -> Tuple[str, bool]:
        original = query
        patterns = [
            (r'<\\|[^>]*\\|>', ''),
            (r'system\\s*:', '', re.IGNORECASE),
            (r'ignore\\s+(previous|all|above)', '', re.IGNORECASE),
        ]
        cleaned = query
        for pattern, replacement, *flags in patterns:
            flag = flags[0] if flags else 0
            cleaned = re.sub(pattern, replacement, cleaned, flags=flag)
        return cleaned.strip(), (cleaned != original)


class LLMJudge:
    def __init__(self, llm):
        self.llm = llm

    def evaluate_response(self, query: str, context: str, response: str) -> Dict:
        if len(context) > 2000:
            context = context[:2000] + "..."

        prompt = f"""Evaluate this RAG response (scale 1-5):

CONTEXT: {{context[:1500]}}
QUESTION: {{query}}
ANSWER: {{response}}

Evaluate:
1. FAITHFULNESS: 1-5
2. RELEVANCE: 1-5
3. COMPLETENESS: 1-5

Respond ONLY with JSON:
{{{{"faithfulness": N, "relevance": N, "completeness": N, "reasoning": "max 80 char"}}}}"""

        try:
            result = self.llm.invoke(prompt)
            json_str = result.content if hasattr(result, 'content') else str(result)
            json_str = re.sub(r'```json\\s*', '', json_str)
            json_str = re.sub(r'```\\s*$', '', json_str)
            scores = json.loads(json_str.strip())
            scores['average'] = round((scores['faithfulness'] + scores['relevance'] + scores['completeness']) / 3, 2)
            return scores
        except Exception as e:
            return {{'faithfulness': 0, 'relevance': 0, 'completeness': 0, 'average': 0, 'reasoning': str(e)[:50]}}


def augment_query_for_images(query: str, llm_instance) -> Tuple[str, str]:
    augment_prompt = f"""Given this automotive manual question:
"{{query}}"

Generate 5-8 keywords for finding relevant images. Include: components, positions, actions.
Respond ONLY with keywords separated by spaces.
Example: button dashboard display reset lever"""

    try:
        result = llm_instance.invoke(augment_prompt)
        keywords = result.content if hasattr(result, 'content') else str(result)
        keywords = keywords.strip().replace('\\n', ' ').replace(',', ' ')
        keywords = re.sub(r'[^a-zA-Z\\s]', '', keywords)
        keywords = ' '.join(keywords.split())
        augmented = f"{{query}} {{keywords}}"
        return augmented, keywords
    except Exception as e:
        logger.warning(f"Augmentation failed: {{e}}")
        return query, ""


@dataclass
class QueryLogEntry:
    timestamp: str
    query: str
    response: str
    vehicle: Optional[str]
    confidence: Dict
    llm_judge: Dict
    retrieval_metrics: Dict
    performance: Dict
    clip_used: bool
    injection_detected: bool
    images_found: int
    images_shown: int
    language: str


class EvaluationLogger:
    def __init__(self):
        self.queries_log = QUERIES_LOG
        self.report_path = REPORT_PATH
        self.queries_log.parent.mkdir(parents=True, exist_ok=True)
        self.report_path.parent.mkdir(parents=True, exist_ok=True)

    def log_query(self, data: Dict):
        entry = QueryLogEntry(
            timestamp=datetime.now().isoformat(),
            query=data.get('query', ''),
            response=data.get('response', ''),
            vehicle=data.get('vehicle'),
            confidence=data.get('confidence', {{}}),
            llm_judge=data.get('llm_judge', {{}}),
            retrieval_metrics=data.get('retrieval_metrics', {{}}),
            performance=data.get('performance', {{}}),
            clip_used=data.get('clip_used', False),
            injection_detected=data.get('injection_detected', False),
            images_found=data.get('images_found', 0),
            images_shown=data.get('images_shown', 0),
            language=data.get('language', 'unknown')
        )

        with open(self.queries_log, 'a', encoding='utf-8') as f:
            f.write(json.dumps(asdict(entry), ensure_ascii=False) + '\\n')

        self.generate_markdown_report()

    def load_all_queries(self) -> List[Dict]:
        if not self.queries_log.exists():
            return []
        queries = []
        with open(self.queries_log, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    queries.append(json.loads(line))
                except:
                    continue
        return queries

    def generate_markdown_report(self) -> str:
        queries = self.load_all_queries()

        if not queries:
            report = "# AutoMATE Evaluation Report\\n\\n**No data yet.**"
            with open(self.report_path, 'w', encoding='utf-8') as f:
                f.write(report)
            return str(self.report_path)

        confidences = [q.get('confidence', {{}}).get('score', 0) for q in queries]
        judge_scores = [q.get('llm_judge', {{}}).get('average', 0) for q in queries if q.get('llm_judge', {{}}).get('average', 0) > 0]

        ret_div = [q.get('retrieval_metrics', {{}}).get('diversity', 0) for q in queries]
        ret_con = [q.get('retrieval_metrics', {{}}).get('consistency', 0) for q in queries]
        ret_sim = [q.get('retrieval_metrics', {{}}).get('avg_similarity', 0) for q in queries]

        perf_tot = [q.get('performance', {{}}).get('total_time_ms', 0) for q in queries]
        perf_rag = [q.get('performance', {{}}).get('rag_time_ms', 0) for q in queries]
        perf_llm = [q.get('performance', {{}}).get('llm_time_ms', 0) for q in queries]
        perf_img = [q.get('performance', {{}}).get('img_time_ms', 0) for q in queries]

        last_10 = queries[-10:]

        report = f"""# AutoMATE Evaluation Report
**Automotive Multimodal Augmented Technical Expert**

*Auto-generated: {{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}}*

---

## 📊 Executive Summary

- **Total Queries**: {{len(queries)}}
- **Period**: {{queries[0]['timestamp'][:10]}} to {{queries[-1]['timestamp'][:10]}}

---

## 🎯 Aggregate Metrics

### Confidence

| Metric | Value |
|--------|-------|
| Mean | {{np.mean(confidences):.3f}} ± {{np.std(confidences):.3f}} |
| Range | [{{min(confidences):.3f}}, {{max(confidences):.3f}}] |

### LLM Judge ({{len(judge_scores)}} samples)

| Metric | Value |
|--------|-------|
| Mean | {{np.mean(judge_scores) if judge_scores else 0:.3f}}/5.0 ± {{np.std(judge_scores) if judge_scores else 0:.3f}} |

### Retrieval

| Metric | Value |
|--------|-------|
| Diversity | {{np.mean(ret_div):.3f}} ± {{np.std(ret_div):.3f}} |
| Consistency | {{np.mean(ret_con):.3f}} ± {{np.std(ret_con):.3f}} |
| Avg Similarity | {{np.mean(ret_sim):.3f}} ± {{np.std(ret_sim):.3f}} |

### Performance

| Metric | Value |
|--------|-------|
| Total Time | {{np.mean(perf_tot):.0f}}ms |
| RAG Time | {{np.mean(perf_rag):.0f}}ms |
| LLM Time | {{np.mean(perf_llm):.0f}}ms |
| Image Time | {{np.mean(perf_img):.0f}}ms |

---

## 🔍 Last 10 Queries

"""

        for idx, q in enumerate(reversed(last_10), 1):
            qtext = q.get('query', 'N/A')
            vehicle = q.get('vehicle', 'Generic')
            lang = q.get('language', '?')

            conf = q.get('confidence', {{}})
            conf_score = conf.get('score', 0)
            conf_label = conf.get('label', 'N/A')
            conf_bd = conf.get('breakdown', {{}})

            judge = q.get('llm_judge', {{}})
            ret = q.get('retrieval_metrics', {{}})
            perf = q.get('performance', {{}})

            report += f"""### Query #{{len(queries) - len(last_10) + idx}}: {{qtext[:70]}}{{"..." if len(qtext) > 70 else ""}}

**Vehicle:** {{vehicle}} | **Language:** {{lang}}

#### Confidence: {{conf_label}} ({{conf_score}})

| Component | Score |
|-----------|-------|
| Retrieval | {{conf_bd.get('retrieval', 0)}} |
| Relevance | {{conf_bd.get('relevance', 0)}} |
| Quality | {{conf_bd.get('quality', 0)}} |

#### Retrieval Metrics

| Metric | Value |
|--------|-------|
| Diversity | {{ret.get('diversity', 0)}} |
| Consistency | {{ret.get('consistency', 0)}} |
| Avg Similarity | {{ret.get('avg_similarity', 0)}} |
| Chunks | {{ret.get('num_chunks', 0)}} |

"""

            if judge.get('average', 0) > 0:
                report += f"""#### LLM Judge: {{judge['average']}}/5

| Dimension | Score |
|-----------|-------|
| Faithfulness | {{judge.get('faithfulness', 0)}}/5 |
| Relevance | {{judge.get('relevance', 0)}}/5 |
| Completeness | {{judge.get('completeness', 0)}}/5 |

"""

            report += f"""#### Performance

Total: {{perf.get('total_time_ms', 0):.0f}}ms | RAG: {{perf.get('rag_time_ms', 0):.0f}}ms | LLM: {{perf.get('llm_time_ms', 0):.0f}}ms | Images: {{perf.get('img_time_ms', 0):.0f}}ms

#### Images: {{q.get('images_shown', 0)}}/{{q.get('images_found', 0)}} shown

---

"""

        report += "\\n*Auto-generated by AutoMATE v3.5*\\n"

        with open(self.report_path, 'w', encoding='utf-8') as f:
            f.write(report)

        return str(self.report_path)


# ==========================================
# RESOURCE LOADING
# ==========================================

@st.cache_resource
def load_resources():
    with st.spinner("Loading resources..."):
        text_embed = HuggingFaceEmbeddings(model_name=TEXT_EMBED_MODEL)

        text_db = Chroma(
            collection_name=TEXT_COLLECTION_NAME,
            persist_directory=CHROMA_TEXT_DIR,
            embedding_function=text_embed
        )

        img_db = Chroma(
            collection_name=IMAGE_COLLECTION_NAME,
            persist_directory=CHROMA_IMAGE_DIR,
            embedding_function=text_embed
        )

        with open(CLIP_JSON_PATH, 'r') as f:
            clip_embeddings = json.load(f)

        clip_model = CLIPModel.from_pretrained(CLIP_MODEL_NAME).to('cuda')
        clip_processor = CLIPProcessor.from_pretrained(CLIP_MODEL_NAME)

        llm = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash-lite",
            google_api_key=GOOGLE_API_KEY,
            temperature=0.1,
            safety_settings={{
                HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
            }}
        )

        return text_embed, text_db, img_db, clip_embeddings, clip_model, clip_processor, llm


# ==========================================
# MAIN APP
# ==========================================

st.markdown('<div class="main-header">🚗 AutoMATE</div>', unsafe_allow_html=True)
st.markdown('<div class="subtitle">Automotive Multimodal Augmented Technical Expert</div>', unsafe_allow_html=True)
st.markdown("**Vehicle-Aware • Cross-Language • Auto-Evaluation**")

# ==========================================
# SIDEBAR
# ==========================================

with st.sidebar:
    st.header("⚙️ Settings")

    st.subheader("🌐 Language")
    response_language = st.selectbox(
        "Response Language",
        ["Italian", "English", "Auto (detect)"],
        index=1,
        help="Manuals are in Italian. English queries will be auto-translated for retrieval."
    )

    st.markdown("---")

    st.subheader("🖼️ Images")
    enable_augmentation = st.checkbox("Query Augmentation", True, help="Expands query with visual keywords")

    st.markdown("---")

    st.subheader("🔍 Quality Filter")
    enable_quality_filter = st.checkbox("Enable", True, help="Filter out low-quality images")

    if enable_quality_filter:
        min_width = st.slider("Min Width (px)", 0, 200, DEFAULT_MIN_WIDTH, 10)
        min_height = st.slider("Min Height (px)", 0, 200, DEFAULT_MIN_HEIGHT, 10)
        min_size_kb = st.slider("Min Size (KB)", 0.0, 50.0, DEFAULT_MIN_SIZE_KB, 0.5)
        max_aspect = st.slider("Max Ratio", 2.0, 15.0, DEFAULT_MAX_ASPECT_RATIO, 0.5)
    else:
        min_width = 0
        min_height = 0
        min_size_kb = 0.0
        max_aspect = 99.0

    st.markdown("---")

    st.subheader("📊 Retrieval")
    stage1_k = st.slider("Stage 1 (candidates)", 10, 60, STAGE1_TOP_K, 5)
    stage2_k = st.slider("Stage 2 (results)", 1, 10, STAGE2_TOP_K, 1)
    clip_weight = st.slider("CLIP Weight", 0.0, 1.0, DEFAULT_CLIP_WEIGHT, 0.05)

    st.markdown("---")

    enable_judge = st.checkbox("LLM Judge", False, help="Adds 3-5s per query")

    st.markdown("---")

    if st.session_state.resources_loaded:
        st.success("✅ Ready")
    else:
        st.warning("⏳ Loading...")

# ==========================================
# LOAD RESOURCES
# ==========================================

if not st.session_state.resources_loaded:
    try:
        resources = load_resources()
        (st.session_state.text_embed, st.session_state.text_db, st.session_state.img_db,
         st.session_state.clip_embeddings, st.session_state.clip_model,
         st.session_state.clip_processor, st.session_state.llm) = resources
        st.session_state.resources_loaded = True
        st.rerun()
    except Exception as e:
        st.error(f"Error: {{e}}")
        st.stop()

# ==========================================
# QUERY INPUT
# ==========================================

query = st.text_input(
    "🔍 Enter your question:",
    placeholder="e.g., How does the TRIP button work? / Come funziona il pulsante TRIP?"
)

submit = st.button("🚀 Search", type="primary")

# ==========================================
# MAIN LOGIC
# ==========================================

if submit and query:
    start_time = time.time()

    sanitized_query, was_modified = PromptProtection.sanitize_query(query)
    if was_modified:
        st.warning("⚠️ Query sanitized for security")

    vehicle = VehicleDetector.detect(sanitized_query)

    lang_badge = {{"Italian": "🇮🇹", "English": "🇬🇧", "Auto (detect)": "🌐"}}.get(response_language, "🌐")

    col1, col2 = st.columns([3, 1])
    with col1:
        if vehicle:
            st.markdown(f'<span class="vehicle-badge">{{vehicle}}</span>', unsafe_allow_html=True)
    with col2:
        st.markdown(f'<span class="lang-badge">{{lang_badge}}</span>', unsafe_allow_html=True)

    # ==========================================
    # TEXT RAG (OPTIMIZED FOR ITALIAN MANUALS)
    # ==========================================

    rag_start = time.time()

    with st.spinner("Processing RAG..."):

        # ✅ CROSS-LANGUAGE STRATEGY: Manuals are in Italian
        retrieval_query = sanitized_query
        needs_translation = False
        translation_time = 0

        # Detect if query needs translation
        if response_language == "English":
            needs_translation = True
        elif response_language == "Auto (detect)":
            # Simple language detection
            en_indicators = ['how', 'what', 'does', 'work', 'button', 'display', 'when', 'where', 'the']
            it_indicators = ['come', 'cosa', 'funziona', 'pulsante', 'display', 'quando', 'dove', 'il']

            query_lower = sanitized_query.lower()
            en_count = sum(1 for w in en_indicators if w in query_lower)
            it_count = sum(1 for w in it_indicators if w in query_lower)

            needs_translation = (en_count > it_count)

        # Translate query for retrieval if needed
        if needs_translation:
            translation_start = time.time()

            translate_prompt = f"""Translate this automotive technical question to Italian.
Keep technical terms precise and maintain context.

English: {{sanitized_query}}

Italian translation (ONLY the translation, no explanations):"""

            try:
                translation_result = st.session_state.llm.invoke(translate_prompt)
                retrieval_query = translation_result.content if hasattr(translation_result, 'content') else str(translation_result)
                retrieval_query = retrieval_query.strip()

                translation_time = (time.time() - translation_start) * 1000

                # Show translation
                with st.expander("🔄 Query Translation (for retrieval)", expanded=False):
                    st.caption("Manuals are in Italian. Query translated for better retrieval accuracy.")
                    st.write(f"**Original:** {{sanitized_query}}")
                    st.write(f"**Translated:** {{retrieval_query}}")
                    st.caption(f"⏱️ Translation: {{translation_time:.0f}}ms")

            except Exception as e:
                st.warning(f"⚠️ Translation failed: {{e}}. Using original query (accuracy may be reduced).")
                retrieval_query = sanitized_query

        # Retrieval with (possibly translated) query
        retrieval_start = time.time()

        if vehicle:
            docs_with_scores = st.session_state.text_db.similarity_search_with_score(
                retrieval_query, k=18, filter={{"manual": vehicle}}
            )[:6]
        else:
            docs_with_scores = st.session_state.text_db.similarity_search_with_score(retrieval_query, k=6)

        retrieved_docs = [doc for doc, _ in docs_with_scores]

        retrieval_elapsed = (time.time() - retrieval_start) * 1000

        # Retrieval quality check
        if len(docs_with_scores) > 0:
            avg_similarity = np.mean([1 - score for _, score in docs_with_scores])

            with st.expander("📊 Retrieval Quality", expanded=False):
                st.write(f"**Chunks retrieved:** {{len(retrieved_docs)}}")
                st.write(f"**Avg similarity:** {{avg_similarity:.3f}}")

                if avg_similarity < 0.5:
                    st.warning("⚠️ Low retrieval quality. Results may be less accurate.")
                elif avg_similarity > 0.75:
                    st.success("✅ High retrieval quality!")

                st.write(f"**Retrieval time:** {{retrieval_elapsed:.0f}}ms")

                # Preview top 3 chunks
                st.caption("**Top 3 chunks:**")
                for i, (doc, score) in enumerate(docs_with_scores[:3], 1):
                    sim = 1 - score
                    snippet = doc.page_content[:80].replace('\\n', ' ')
                    st.text(f"{{i}}. [{{sim:.3f}}] {{snippet}}...")

        context_parts = [f"[Section {{i+1}}]\\n{{doc.page_content.strip()}}" for i, doc in enumerate(retrieved_docs)]
        context_text = "\\n\\n---\\n\\n".join(context_parts)

        # ✅ CROSS-LANGUAGE PROMPTS
        if response_language == "English":
            template = """You are an expert automotive technician.

IMPORTANT: The manual content below is in ITALIAN, but you must answer in ENGLISH.

INSTRUCTIONS:
1. Carefully read and understand the Italian technical documentation
2. Answer ONLY using information from the provided context
3. Translate technical procedures and specifications accurately to English
4. If information is not in the context, clearly state: "This information is not available in the manual"
5. Maintain technical precision and include all relevant details (values, warnings, procedures)

CONTEXT (Italian manual):
{{context}}

USER QUESTION (English):
{{question}}

TECHNICAL ANSWER (English):"""

        elif response_language == "Auto (detect)":
            template = """You are an expert automotive technician.

INSTRUCTIONS:
1. Answer ONLY using information from the CONTEXT
2. If information is not in the context, clearly state so
3. Include ALL technical details: procedures, values, warnings
4. Respond in the SAME LANGUAGE as the question (Italian or English)

CONTEXT:
{{context}}

QUESTION:
{{question}}

TECHNICAL ANSWER:"""

        else:  # Italian
            template = """Sei un tecnico esperto automotive.

ISTRUZIONI:
1. Rispondi SOLO usando informazioni dal CONTESTO fornito
2. Se l'informazione non è presente nel contesto, dillo chiaramente
3. Includi TUTTI i dettagli tecnici: procedure, valori, avvertenze
4. Rispondi in italiano

CONTESTO:
{{context}}

DOMANDA:
{{question}}

RISPOSTA TECNICA:"""

        prompt = PromptTemplate(template=template, input_variables=["context", "question"])

        chain = (
            {{
                "context": RunnableLambda(lambda _: context_text),
                "question": RunnablePassthrough()
            }}
            | prompt
            | st.session_state.llm
            | StrOutputParser()
        )

        # Generate with ORIGINAL query (not translated)
        llm_start = time.time()
        response = chain.invoke(sanitized_query)
        llm_time = (time.time() - llm_start) * 1000

        # Confidence metrics
        ret_conf = ConfidenceCalculator.retrieval_confidence(docs_with_scores)
        rel_conf = ConfidenceCalculator.context_relevance(sanitized_query, retrieved_docs, st.session_state.text_embed)
        qual_conf = ConfidenceCalculator.answer_quality(response)
        confidence = ConfidenceCalculator.aggregate(ret_conf, rel_conf, qual_conf)

    rag_time = (time.time() - rag_start) * 1000

    # ==========================================
    # DISPLAY RESPONSE
    # ==========================================

    st.markdown("### 📖 Answer")
    st.write(response)

    conf_class = f"confidence-{{confidence['label'].lower()}}"
    st.markdown(f'<div class="{{conf_class}}">🎯 {{confidence["label"]}} ({{confidence["score"]}})</div>', unsafe_allow_html=True)

    c1, c2, c3 = st.columns(3)
    with c1:
        st.metric("🔍 Retrieval", f"{{confidence['breakdown']['retrieval']}}")
    with c2:
        st.metric("🎯 Relevance", f"{{confidence['breakdown']['relevance']}}")
    with c3:
        st.metric("✍️ Quality", f"{{confidence['breakdown']['quality']}}")

    # ==========================================
    # LLM JUDGE
    # ==========================================

    judge_data = {{}}
    if enable_judge:
        with st.spinner("🤖 LLM Judge evaluation..."):
            judge = LLMJudge(st.session_state.llm)
            judge_data = judge.evaluate_response(sanitized_query, context_text, response)

        st.markdown("### 🤖 LLM Judge")
        j1, j2, j3 = st.columns(3)
        with j1:
            st.metric("Faithfulness", f"{{judge_data.get('faithfulness', 0)}}/5")
        with j2:
            st.metric("Relevance", f"{{judge_data.get('relevance', 0)}}/5")
        with j3:
            st.metric("Completeness", f"{{judge_data.get('completeness', 0)}}/5")

        st.caption(f"Average: {{judge_data.get('average', 0):.2f}}/5 | {{judge_data.get('reasoning', 'N/A')}}")

    # ==========================================
    # IMAGE RETRIEVAL
    # ==========================================

    img_start = time.time()

    with st.spinner("🖼️ Searching images..."):
        augmented_keywords = ""
        if enable_augmentation:
            image_query, augmented_keywords = augment_query_for_images(sanitized_query, st.session_state.llm)
        else:
            image_query = sanitized_query

        # Stage 1: Text similarity
        if vehicle:
            img_candidates = st.session_state.img_db.similarity_search_with_score(
                image_query, k=stage1_k*3, filter={{"manual": vehicle}}
            )[:stage1_k]
        else:
            img_candidates = st.session_state.img_db.similarity_search_with_score(image_query, k=stage1_k)

        # Stage 2: CLIP re-ranking
        inputs = st.session_state.clip_processor(text=[image_query], return_tensors="pt", padding=True)
        inputs = {{k: v.to('cuda') for k, v in inputs.items()}}

        with torch.no_grad():
            query_features = st.session_state.clip_model.get_text_features(**inputs)
            query_features = query_features / query_features.norm(dim=-1, keepdim=True)

        query_emb = query_features[0].cpu().numpy()

        reranked = []
        for doc, text_score in img_candidates:
            img_id = doc.metadata.get('image_id')
            has_clip = doc.metadata.get('has_clip', False)

            if has_clip and img_id in st.session_state.clip_embeddings:
                clip_emb = np.array(st.session_state.clip_embeddings[img_id])
                clip_score = np.dot(query_emb, clip_emb) / (np.linalg.norm(query_emb) * np.linalg.norm(clip_emb))
                text_sim = 1 - text_score
                combined = (clip_weight * clip_score) + ((1 - clip_weight) * text_sim)
                reranked.append((doc, combined, True))
            else:
                reranked.append((doc, 1 - text_score, False))

        reranked.sort(key=lambda x: x[1], reverse=True)
        top_imgs = reranked[:stage2_k]

        # Quality filter
        quality_threshold = ImageQualityThreshold(min_width, min_height, min_size_kb, max_aspect)
        filtered_imgs = ImageQualityFilter.filter_results(top_imgs, quality_threshold, enable_quality_filter)

        del query_features, query_emb, inputs
        gc.collect()
        torch.cuda.empty_cache()

    img_time = (time.time() - img_start) * 1000

    # ==========================================
    # DISPLAY IMAGES
    # ==========================================

    st.markdown("### 🖼️ Relevant Images")

    if augmented_keywords and enable_augmentation:
        with st.expander("🔧 Augmented Keywords", expanded=False):
            st.code(augmented_keywords)

    rejected = len(top_imgs) - len(filtered_imgs)
    st.caption(f"Showing {{len(filtered_imgs)}}/{{stage2_k}} images ({{rejected}} filtered by quality)")

    if filtered_imgs:
        cols = st.columns(min(len(filtered_imgs), 5))
        for idx, (doc, score, used_clip) in enumerate(filtered_imgs):
            with cols[idx % 5]:
                img_path = doc.metadata.get('image_path')
                if img_path and os.path.exists(img_path):
                    try:
                        img = PILImage.open(img_path)
                        w, h = img.size
                        method = "CLIP" if used_clip else "Text"
                        st.image(img, caption=f"#{{idx+1}} [{{method}}]\\n{{score:.3f}} | {{w}}x{{h}}px", use_container_width=True)
                    except:
                        st.warning("Error loading image")
    else:
        if enable_quality_filter:
            st.info("No images passed quality filter. Try disabling it or adjusting thresholds.")
        else:
            st.info("No images found for this query.")

    # ==========================================
    # PERFORMANCE SUMMARY
    # ==========================================

    total_time = (time.time() - start_time) * 1000

    st.caption(f"⏱️ Total: {{total_time:.0f}}ms (RAG: {{rag_time:.0f}}ms | LLM: {{llm_time:.0f}}ms | Images: {{img_time:.0f}}ms)")

    # ==========================================
    # EVALUATION LOGGING
    # ==========================================

    try:
        eval_logger = EvaluationLogger()

        scores_only = [score for _, score in docs_with_scores]
        retrieval_metrics = {{
            'diversity': RetrievalMetrics.diversity_score(retrieved_docs),
            'consistency': RetrievalMetrics.chunk_similarity_variance(scores_only),
            'avg_similarity': RetrievalMetrics.average_chunk_score(scores_only),
            'num_chunks': len(retrieved_docs)
        }}

        eval_logger.log_query({{
            'query': query,
            'response': response,
            'vehicle': vehicle,
            'confidence': confidence,
            'llm_judge': judge_data,
            'retrieval_metrics': retrieval_metrics,
            'performance': {{
                'total_time_ms': total_time,
                'rag_time_ms': rag_time,
                'llm_time_ms': llm_time,
                'img_time_ms': img_time
            }},
            'clip_used': any(used_clip for _, _, used_clip in filtered_imgs),
            'injection_detected': was_modified,
            'images_found': len(top_imgs),
            'images_shown': len(filtered_imgs),
            'language': response_language
        }})

        st.success(f"✅ Evaluation report updated")

    except Exception as e:
        st.warning(f"⚠️ Evaluation logging error: {{e}}")

    # History
    st.session_state.query_history.append({{
        'query': query[:50],
        'vehicle': vehicle,
        'confidence': confidence['score'],
        'images': len(filtered_imgs),
        'language': response_language
    }})

# ==========================================
# FOOTER & HISTORY
# ==========================================

st.markdown("---")
st.caption("AutoMATE v3.5 | Automotive Multimodal Augmented Technical Expert • Cross-Language Optimized")

if st.session_state.query_history:
    with st.sidebar:
        st.markdown("---")
        st.subheader("📜 Recent Queries")
        for idx, entry in enumerate(reversed(st.session_state.query_history[-5:]), 1):
            lang_icon = {{"Italian": "🇮🇹", "English": "🇬🇧", "Auto (detect)": "🌐"}}.get(entry.get('language', ''), '')
            st.caption(f"{{idx}}. {{lang_icon}} {{entry['query']}}... ({{entry['confidence']:.2f}})")
'''

# ==========================================
# SAVE APP
# ==========================================

app_path = "/content/app.py"

with open(app_path, 'w', encoding='utf-8') as f:
    f.write(app_code)

print(f"✅ app.py generated: {app_path}")
print(f"   • Size: {os.path.getsize(app_path) / 1024:.1f}KB")
print(f"   • Features: Cross-language support + Query translation")

# ==========================================
# START STREAMLIT + NGROK
# ==========================================

print("\n" + "="*70)
print("🚀 STARTING STREAMLIT + NGROK")
print("="*70)

from pyngrok import ngrok, conf
import getpass
import subprocess
import threading

try:
    ngrok_token = getpass.getpass("🔑 Ngrok Auth Token: ")
    conf.get_default().auth_token = ngrok_token

    print("\n🔄 Starting Streamlit...")

    !pkill -9 streamlit 2>/dev/null

    def run_streamlit():
        subprocess.run([
            "streamlit", "run", app_path,
            "--server.port=8501",
            "--server.headless=true",
            "--server.fileWatcherType=none",
            "--browser.gatherUsageStats=false"
        ])

    streamlit_thread = threading.Thread(target=run_streamlit, daemon=True)
    streamlit_thread.start()

    import time
    time.sleep(5)

    print("🌐 Creating ngrok tunnel...")
    public_url = ngrok.connect(8501)

    print("\n" + "="*70)
    print("🎉 AutoMATE IS ONLINE!")
    print("="*70)
    print(f"\n🔗 URL: {public_url}")
    print(f"""
✅ AutoMATE v3.5 - Italian Manuals Optimized

🌐 CROSS-LANGUAGE FEATURES:
   • English queries → Auto-translated to Italian for retrieval
   • Italian context → Accurate English responses
   • Auto-detect mode for mixed language usage
   • Translation transparency (see expander for details)

📊 EVALUATION:
   • Auto-updating report after each query
   • Retrieval quality monitoring
   • Full metrics tracking (confidence, LLM judge, performance)
   • Report: /content/drive/MyDrive/OCR/evaluation/reports/automate_evaluation_report.md

💡 USAGE:
   • Italian query: "Come funziona il pulsante TRIP?"
     → Direct retrieval, Italian response

   • English query: "How does the TRIP button work?"
     → Auto-translated to Italian for retrieval
     → Italian context used to generate English response

   • Auto mode: Detects language and responds accordingly

💾 App stays online while notebook runs
""")
    print("="*70)

except Exception as e:
    print(f"\n❌ Error: {e}")
    print("\n🔧 Troubleshooting:")
    print("   1. Get Ngrok token: https://dashboard.ngrok.com/get-started/your-authtoken")
    print("   2. Restart runtime and try again")

# NGROK TOKEN
#NGROK_AUTH_TOKEN = "SAVE_HERE_YOUR_NGROK_TOKEN"